# Identification des variables de contrôle 

( relatif à la partie ***3.4 -Les variables de controle*** du rapport) 

L'objectif de ce notebook est de créer un Dataframe identifiant une ligne par établissement de santé, lié aux variables de contrôle : 
- Idenfitication de l'établissement 
    - Le statut juridique de l'établissement 
    - Le département 
    - La commune 
- Le capital : 
    - Nombre de lits 
    - Matériel & Equipement (Bloc, imagerie, examens biologique)
- Environnement
    - taux de chomage dans le département 
    - couleur politique dans la commune 
    - Indicateurs de richesse       

- Les séjours :
    - Nombre de séjours |

In [1]:
# Importation des packages 
import os 
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
# Les bases : 

# Bases de la SAE : 
path_SAE = 'C:/Users/louis/OneDrive/Documents/ENSAE/2A/Stat_app/SAE_2019/Bases_statistiques/Bases_CSV/'

ID=pd.read_csv(path_SAE+'ID_2019r.csv' ,sep=';', encoding='latin-1')
MCO=pd.read_csv(path_SAE+'MCO_2019r.csv' ,sep=';', encoding='latin-1')
BLOC=pd.read_csv(path_SAE+ 'BLOCS_2019r.csv' ,sep=';', encoding='latin-1')
IMAGES = pd.read_csv(path_SAE+ 'IMAGES_2019r.csv' ,sep=';', encoding='latin-1')
BIO = pd.read_csv(path_SAE+ 'BIO_2019r.csv' ,sep=';', encoding='latin-1')
TELEMED =  pd.read_csv(path_SAE+ 'TELEMED_2019r.csv' ,sep=';', encoding='latin-1')

# Base créée :
path_ISA= r'C:\Users\louis\OneDrive\Documents\ENSAE\2A\Stat_app\ISA.csv'
ISA=pd.read_csv(path_ISA ,sep=',')


# Bases complémentaires :
# Indicateurs de richesse :
path_indicateurs= 'C:/Users/louis/OneDrive/Documents/ENSAE/2A/Stat_app/Bases_complementaires/MDB-INSEE-V2.xls'
indicateurs_richesse = pd.read_excel(path_indicateurs)

  #Taux de chomage : https://www.insee.fr/fr/statistiques/2134411
path_tx_chomage=r'C:\Users\louis\OneDrive\Documents\ENSAE\2A\Stat_app\Bases_complementaires\TCRD_087.xls'
tx_chomage = pd.read_excel(path_tx_chomage)

 # Partis politiques : https://www.data.gouv.fr/fr/datasets/election-presidentielle-des-23-avril-et-7-mai-2017-resultats-definitifs-du-1er-tour-par-communes/   
path_presidentielles = 'C:/Users/louis/OneDrive/Documents/ENSAE/2A/Stat_app/Bases_complementaires/Presidentielle_2017_Resultats_Communes_Tour_1_c.xls'
presidentielles = pd.read_excel(path_presidentielles, skiprows= 3)

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


In [3]:
ISA

,Numero_finess,Points_ISA,stjr
0,010008407,165054.798333,1
1,010009132,5715.170000,1
2,010780054,787929.068333,1
3,010780062,188174.950000,1
4,010780096,90756.543333,1
...,...,...,...
1188,970462024,20018.590000,2
1189,970462073,81023.520000,2
1190,970462081,92668.675000,2
1191,970462107,306221.841667,2


## Identification de l'établissement 

### Le statut juridique 

Le statut juridique est déja présent dans la base ISA puisque qu'il a été utile pour la création de la variable d'interet ( tarifs GHS différents pour le public et le privé) 

### Le département 

In [4]:
# Code département présent dans la base ID 
ID['dep']

0       01
1       01
2       01
3       01
4       01
        ..
4013    9F
4014    9F
4015    9F
4016    9F
4017    9F
Name: dep, Length: 4018, dtype: object

In [5]:
dep_fi = pd.merge (ISA, ID[['fi','dep']], left_on='Numero_finess', right_on='fi',how='inner')
dep_fi

,Numero_finess,Points_ISA,stjr,fi,dep
0,060000528,356892.030000,3,060000528,06
1,060780491,372410.435000,3,060780491,06
2,060780947,174552.540000,3,060780947,06
3,060791811,146432.633333,3,060791811,06
4,060794013,238173.260000,3,060794013,06
...,...,...,...,...,...
663,970462024,20018.590000,2,970462024,9D
664,970462073,81023.520000,2,970462073,9D
665,970462081,92668.675000,2,970462081,9D
666,970462107,306221.841667,2,970462107,9D


In [6]:
dep_fi.fi.nunique()

668

Le numéro finess est bien unique

In [7]:
dep_fiej = pd.merge (ISA, ID[['fi_ej','dep']], left_on='Numero_finess', right_on='fi_ej',how='inner')
dep_fiej

,Numero_finess,Points_ISA,stjr,fi_ej,dep
0,010008407,1.650548e+05,1,010008407,01
1,010008407,1.650548e+05,1,010008407,01
2,010008407,1.650548e+05,1,010008407,01
3,010009132,5.715170e+03,1,010009132,01
4,010009132,5.715170e+03,1,010009132,01
...,...,...,...,...,...
1084,970408589,2.697187e+06,1,970408589,9D
1085,970408589,2.697187e+06,1,970408589,9D
1086,970408589,2.697187e+06,1,970408589,9D
1087,970421038,5.785114e+05,1,970421038,9D


In [8]:
dep_fiej.fi_ej.nunique()

525

Le numéro finess de l'entité juridique n'est pas unique. On supprime les doubes 

In [9]:
dep_fiej.drop_duplicates(subset ='fi_ej', keep = 'first', inplace=True)

In [10]:
dep_fiej

,Numero_finess,Points_ISA,stjr,fi_ej,dep
0,010008407,1.650548e+05,1,010008407,01
3,010009132,5.715170e+03,1,010009132,01
5,010780054,7.879291e+05,1,010780054,01
8,010780062,1.881749e+05,1,010780062,01
9,010780096,9.075654e+04,1,010780096,01
...,...,...,...,...,...
1078,970305629,9.421212e+04,1,970305629,9C
1079,970403606,3.325696e+05,1,970403606,9D
1081,970408589,2.697187e+06,1,970408589,9D
1087,970421038,5.785114e+05,1,970421038,9D


In [11]:
Var_CTRL = pd.concat([dep_fi, dep_fiej])
Var_CTRL

,Numero_finess,Points_ISA,stjr,fi,dep,fi_ej
0,060000528,3.568920e+05,3,060000528,06,NaN
1,060780491,3.724104e+05,3,060780491,06,NaN
2,060780947,1.745525e+05,3,060780947,06,NaN
3,060791811,1.464326e+05,3,060791811,06,NaN
4,060794013,2.381733e+05,3,060794013,06,NaN
...,...,...,...,...,...,...
1078,970305629,9.421212e+04,1,NaN,9C,970305629
1079,970403606,3.325696e+05,1,NaN,9D,970403606
1081,970408589,2.697187e+06,1,NaN,9D,970408589
1087,970421038,5.785114e+05,1,NaN,9D,970421038


In [12]:
(ISA.shape[0],Var_CTRL.shape[0] )

(1193, 1193)

On a bien récupéré l'ensemble de nos hopitaux  

### Commune 

In [13]:
# Code commune présent dans la base ID 
ID['COMINSEE']

0       01451
1       01034
2       01427
3       01173
4       01244
        ...  
4013    9F611
4014    9F611
4015    9F611
4016    9F606
4017    9F606
Name: COMINSEE, Length: 4018, dtype: object

In [14]:
com_fi = pd.merge (Var_CTRL, ID[['fi','COMINSEE']], left_on='Numero_finess', right_on='fi',how='inner')
com_fi

,Numero_finess,Points_ISA,stjr,fi_x,dep,fi_ej,fi_y,COMINSEE
0,060000528,356892.030000,3,060000528,06,NaN,060000528,06088
1,060780491,372410.435000,3,060780491,06,NaN,060780491,06123
2,060780947,174552.540000,3,060780947,06,NaN,060780947,06088
3,060791811,146432.633333,3,060791811,06,NaN,060791811,06088
4,060794013,238173.260000,3,060794013,06,NaN,060794013,06123
...,...,...,...,...,...,...,...,...
663,970462024,20018.590000,2,970462024,9D,NaN,970462024,9D407
664,970462073,81023.520000,2,970462073,9D,NaN,970462073,9D422
665,970462081,92668.675000,2,970462081,9D,NaN,970462081,9D407
666,970462107,306221.841667,2,970462107,9D,NaN,970462107,9D411


In [15]:
com_fiej = pd.merge (Var_CTRL, ID[['fi_ej','COMINSEE']], left_on='Numero_finess', right_on='fi_ej',how='inner')
com_fiej

,Numero_finess,Points_ISA,stjr,fi,dep,fi_ej_x,fi_ej_y,COMINSEE
0,010008407,1.650548e+05,1,NaN,01,010008407,010008407,01283
1,010008407,1.650548e+05,1,NaN,01,010008407,010008407,01269
2,010008407,1.650548e+05,1,NaN,01,010008407,010008407,01283
3,010009132,5.715170e+03,1,NaN,01,010009132,010009132,01306
4,010009132,5.715170e+03,1,NaN,01,010009132,010009132,01420
...,...,...,...,...,...,...,...,...
1084,970408589,2.697187e+06,1,NaN,9D,970408589,970408589,9D424
1085,970408589,2.697187e+06,1,NaN,9D,970408589,970408589,9D412
1086,970408589,2.697187e+06,1,NaN,9D,970408589,970408589,9D422
1087,970421038,5.785114e+05,1,NaN,9D,970421038,970421038,9D415


In [16]:
com_fiej.drop_duplicates(subset ='fi_ej_x', keep = 'first', inplace=True)

In [17]:
com_fiej

,Numero_finess,Points_ISA,stjr,fi,dep,fi_ej_x,fi_ej_y,COMINSEE
0,010008407,1.650548e+05,1,NaN,01,010008407,010008407,01283
3,010009132,5.715170e+03,1,NaN,01,010009132,010009132,01306
5,010780054,7.879291e+05,1,NaN,01,010780054,010780054,01451
8,010780062,1.881749e+05,1,NaN,01,010780062,010780062,01034
9,010780096,9.075654e+04,1,NaN,01,010780096,010780096,01427
...,...,...,...,...,...,...,...,...
1078,970305629,9.421212e+04,1,NaN,9C,970305629,970305629,9C304
1079,970403606,3.325696e+05,1,NaN,9D,970403606,970403606,9D410
1081,970408589,2.697187e+06,1,NaN,9D,970408589,970408589,9D411
1087,970421038,5.785114e+05,1,NaN,9D,970421038,970421038,9D415


In [18]:
Var_CTRL = pd.concat([com_fi, com_fiej])
Var_CTRL

,Numero_finess,Points_ISA,stjr,fi_x,dep,fi_ej,fi_y,COMINSEE,fi,fi_ej_x,fi_ej_y
0,060000528,3.568920e+05,3,060000528,06,NaN,060000528,06088,NaN,NaN,NaN
1,060780491,3.724104e+05,3,060780491,06,NaN,060780491,06123,NaN,NaN,NaN
2,060780947,1.745525e+05,3,060780947,06,NaN,060780947,06088,NaN,NaN,NaN
3,060791811,1.464326e+05,3,060791811,06,NaN,060791811,06088,NaN,NaN,NaN
4,060794013,2.381733e+05,3,060794013,06,NaN,060794013,06123,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1078,970305629,9.421212e+04,1,NaN,9C,NaN,NaN,9C304,NaN,970305629,970305629
1079,970403606,3.325696e+05,1,NaN,9D,NaN,NaN,9D410,NaN,970403606,970403606
1081,970408589,2.697187e+06,1,NaN,9D,NaN,NaN,9D411,NaN,970408589,970408589
1087,970421038,5.785114e+05,1,NaN,9D,NaN,NaN,9D415,NaN,970421038,970421038


In [19]:
Var_CTRL= Var_CTRL.drop(['fi_x','fi_y','fi_ej_x', 'fi_ej_y', 'fi_ej','fi'], axis=1)

## Capital 

### Le nombre de lits 

On souhaite obtenir pour chaque établissement, le nombre de lits associés. 

On commence par fusionner en fonction du numéro finess : 

In [20]:
lits_fi = pd.merge (Var_CTRL, MCO[['FI','LIT_MCO']], left_on='Numero_finess', right_on='FI',how='inner')
lits_fi

,Numero_finess,Points_ISA,stjr,dep,COMINSEE,FI,LIT_MCO
0,060000528,356892.030000,3,06,06088,060000528,128.0
1,060780491,372410.435000,3,06,06123,060780491,128.0
2,060780947,174552.540000,3,06,06088,060780947,105.0
3,060791811,146432.633333,3,06,06088,060791811,91.0
4,060794013,238173.260000,3,06,06123,060794013,68.0
...,...,...,...,...,...,...,...
663,970462024,20018.590000,2,9D,9D407,970462024,38.0
664,970462073,81023.520000,2,9D,9D422,970462073,69.0
665,970462081,92668.675000,2,9D,9D407,970462081,45.0
666,970462107,306221.841667,2,9D,9D411,970462107,266.0


In [21]:
lits_fi.Numero_finess.nunique()

668

Le numero finess est bien unique. 

On fusionne en fonction du numéro finess de l'entité juridique :

In [22]:
lits_fiej = pd.merge (Var_CTRL, MCO[['FI_EJ','LIT_MCO']], left_on='Numero_finess', right_on='FI_EJ',how='inner')
lits_fiej

,Numero_finess,Points_ISA,stjr,dep,COMINSEE,FI_EJ,LIT_MCO
0,010008407,1.650548e+05,1,01,01283,010008407,102.0
1,010009132,5.715170e+03,1,01,01306,010009132,12.0
2,010009132,5.715170e+03,1,01,01306,010009132,8.0
3,010780054,7.879291e+05,1,01,01451,010780054,369.0
4,010780062,1.881749e+05,1,01,01034,010780062,93.0
...,...,...,...,...,...,...,...
743,970408589,2.697187e+06,1,9D,9D411,970408589,47.0
744,970408589,2.697187e+06,1,9D,9D411,970408589,15.0
745,970408589,2.697187e+06,1,9D,9D411,970408589,44.0
746,970421038,5.785114e+05,1,9D,9D415,970421038,276.0


In [23]:
lits_fiej.FI_EJ.nunique()

525

Le numéro finess de l'entité juridique n'est pas unique. On peut afficher ces doublons avant de les traiter :

In [24]:
# Affichage des duplicatas 
lits_fiej[lits_fiej.duplicated(subset ='Numero_finess')]

,Numero_finess,Points_ISA,stjr,dep,COMINSEE,FI_EJ,LIT_MCO
2,010009132,5.715170e+03,1,01,01306,010009132,8.0
25,050002948,6.212138e+05,1,04,04209,050002948,246.0
30,060785011,2.515840e+06,1,06,06088,060785011,576.0
31,060785011,2.515840e+06,1,06,06088,060785011,50.0
32,060785011,2.515840e+06,1,06,06088,060785011,425.0
...,...,...,...,...,...,...,...
736,970211207,1.597206e+06,1,9B,9B209,970211207,147.0
742,970408589,2.697187e+06,1,9D,9D411,970408589,547.0
743,970408589,2.697187e+06,1,9D,9D411,970408589,47.0
744,970408589,2.697187e+06,1,9D,9D411,970408589,15.0


In [25]:
# On effectue la moyenne :
df_fiunique_lits = pd.DataFrame({'Numero_finess':lits_fiej.groupby('FI_EJ', as_index=False).mean()['FI_EJ'], 'FI_EJ':lits_fiej.groupby('FI_EJ', as_index=False).mean()['FI_EJ'], 'stjr':lits_fiej.groupby('FI_EJ', as_index=False).mean()['stjr'],'LIT_MCO': lits_fiej.groupby('FI_EJ', as_index=False).mean()['LIT_MCO'] })
# On retire les doublons 
lits_fiej.drop_duplicates(subset ='Numero_finess', keep = 'first', inplace=True)
# On fusionne
lits_fiej_final = pd.merge (df_fiunique_lits,lits_fiej[['Numero_finess','Points_ISA','dep','COMINSEE']], left_on='Numero_finess', right_on='Numero_finess',how='inner')
lits_fiej_final

,Numero_finess,FI_EJ,stjr,LIT_MCO,Points_ISA,dep,COMINSEE
0,010008407,010008407,1,102.0,1.650548e+05,01,01283
1,010009132,010009132,1,10.0,5.715170e+03,01,01306
2,010780054,010780054,1,369.0,7.879291e+05,01,01451
3,010780062,010780062,1,93.0,1.881749e+05,01,01034
4,010780096,010780096,1,59.0,9.075654e+04,01,01427
...,...,...,...,...,...,...,...
520,970305629,970305629,1,92.0,9.421212e+04,9C,9C304
521,970403606,970403606,1,177.0,3.325696e+05,9D,9D410
522,970408589,970408589,1,245.6,2.697187e+06,9D,9D411
523,970421038,970421038,1,276.0,5.785114e+05,9D,9D415


On fusionne les deux tables (fi_ej et fi) : 

In [26]:
Var_CTRL = pd.concat([lits_fi, lits_fiej_final])
Var_CTRL

,Numero_finess,Points_ISA,stjr,dep,COMINSEE,FI,LIT_MCO,FI_EJ
0,060000528,3.568920e+05,3,06,06088,060000528,128.0,NaN
1,060780491,3.724104e+05,3,06,06123,060780491,128.0,NaN
2,060780947,1.745525e+05,3,06,06088,060780947,105.0,NaN
3,060791811,1.464326e+05,3,06,06088,060791811,91.0,NaN
4,060794013,2.381733e+05,3,06,06123,060794013,68.0,NaN
...,...,...,...,...,...,...,...,...
520,970305629,9.421212e+04,1,9C,9C304,NaN,92.0,970305629
521,970403606,3.325696e+05,1,9D,9D410,NaN,177.0,970403606
522,970408589,2.697187e+06,1,9D,9D411,NaN,245.6,970408589
523,970421038,5.785114e+05,1,9D,9D415,NaN,276.0,970421038


In [27]:
(Var_CTRL.shape,ISA.shape)

((1193, 8), (1193, 3))

In [28]:
Var_CTRL= Var_CTRL.drop(['FI','FI_EJ'], axis=1)

In [29]:
Var_CTRL

,Numero_finess,Points_ISA,stjr,dep,COMINSEE,LIT_MCO
0,060000528,3.568920e+05,3,06,06088,128.0
1,060780491,3.724104e+05,3,06,06123,128.0
2,060780947,1.745525e+05,3,06,06088,105.0
3,060791811,1.464326e+05,3,06,06088,91.0
4,060794013,2.381733e+05,3,06,06123,68.0
...,...,...,...,...,...,...
520,970305629,9.421212e+04,1,9C,9C304,92.0
521,970403606,3.325696e+05,1,9D,9D410,177.0
522,970408589,2.697187e+06,1,9D,9D411,245.6
523,970421038,5.785114e+05,1,9D,9D415,276.0


In [30]:
Var_CTRL= Var_CTRL.rename(columns={'LIT_MCO' : 'Capital_Lit_MCO'})

On a bien récupérer l'ensemble de nos hopitaux 

### Matériel - Equipement 

Dans la SAE les bases relatives aux matériaux et équipements sont : 
- BLOCS Sites opératoires et salles d’intervention 
- IMAGES Imagerie médicale
- BIO Biologie médicale et anatomo-cytopathologie 
- TELEMED Télémédecine 

#### BLOC 

Dans la base BLOC de la SAE, il est possible d'identifier le nombre de salle d'intervention ou d'exploration total. Cette variable regroupe : 
- Nombre de salles destinées à la chirurgie classique, hors obstétrique (hors salles dédiées à la chirurgie ambulatoire)
- Nombre de salles dédiées aux actes d'obstétrique (césariennes, IVG…) (hors salles de naissance et hors salles dédiées à la chirurgie ambulatoire)
- Nombre de salles d'activités interventionnelles sous imagerie (spécialisées ou non avec ou sans anesthésie) hors salles dédiées à la chirurgie ambulatoire
- Nombre de salles dédiées aux endoscopies avec ou sans anesthésie pour tout type d'activité (ambulatoire, hosp. complète, externe) hors salles dédiées à la chirurgie ambulatoire
- Nombre d'autres salles d'intervention et/ou d'exploration (hors celles dédiées à la chirurgie ambulatoire)


In [31]:
BLOC

,BOR,AN,FI,FI_EJ,SITES,SALCHIR,SAL24CHIR,SALOBS,SAL24OBS,SALINT,...,BLOCS_I14,BLOCS_A15,BLOCS_B15,BLOCS_C15,BLOCS_D15,BLOCS_E15,BLOCS_F15,BLOCS_G15,BLOCS_H15,BLOCS_I15
0,BLOCS,2019,010000024,010780054,2.0,9.0,NaN,2.0,2.0,1.0,...,NaN,11.0,69.0,80.0,24.0,NaN,NaN,NaN,NaN,NaN
1,BLOCS,2019,010000032,010780062,1.0,3.0,1.0,NaN,NaN,NaN,...,0.0,16.0,5.0,21.0,3.0,71.0,0.0,0.0,0.0,0.0
2,BLOCS,2019,010005239,010008407,1.0,3.0,1.0,1.0,1.0,NaN,...,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0
3,BLOCS,2019,010007300,010007292,1.0,0.0,0.0,0.0,0.0,0.0,...,NaN,0.0,2.0,2.0,0.0,NaN,NaN,NaN,NaN,NaN
4,BLOCS,2019,010780195,010000156,1.0,6.0,NaN,NaN,NaN,1.0,...,NaN,14.0,130.0,144.0,77.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1029,BLOCS,2019,970462073,970400271,2.0,7.0,NaN,NaN,NaN,NaN,...,NaN,15.0,14.0,29.0,1.0,NaN,NaN,NaN,NaN,NaN
1030,BLOCS,2019,970462081,970400255,1.0,6.0,NaN,NaN,NaN,1.0,...,NaN,18.0,275.0,293.0,275.0,NaN,NaN,NaN,NaN,NaN
1031,BLOCS,2019,970462107,970400305,1.0,6.0,NaN,1.0,NaN,2.0,...,NaN,24.0,308.0,332.0,320.0,NaN,NaN,NaN,NaN,NaN
1032,BLOCS,2019,970466751,970400446,1.0,NaN,0.0,NaN,NaN,NaN,...,NaN,0.0,1.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN


In [32]:
bloc_fi_salto = pd.merge (Var_CTRL, BLOC[['FI','SALTOT']], left_on='Numero_finess', right_on='FI',how='inner')
bloc_fi_salto

,Numero_finess,Points_ISA,stjr,dep,COMINSEE,Capital_Lit_MCO,FI,SALTOT
0,060000528,356892.030000,3,06,06088,128.0,060000528,8.0
1,060780491,372410.435000,3,06,06123,128.0,060780491,14.0
2,060780947,174552.540000,3,06,06088,105.0,060780947,5.0
3,060791811,146432.633333,3,06,06088,91.0,060791811,2.0
4,060794013,238173.260000,3,06,06123,68.0,060794013,5.0
...,...,...,...,...,...,...,...,...
561,970462024,20018.590000,2,9D,9D407,38.0,970462024,2.0
562,970462073,81023.520000,2,9D,9D422,69.0,970462073,9.0
563,970462081,92668.675000,2,9D,9D407,45.0,970462081,8.0
564,970462107,306221.841667,2,9D,9D411,266.0,970462107,11.0


In [33]:
bloc_fi_ej_salto = pd.merge (Var_CTRL, BLOC[['FI_EJ','SALTOT']], left_on='Numero_finess', right_on='FI_EJ',how='inner')
bloc_fi_ej_salto

,Numero_finess,Points_ISA,stjr,dep,COMINSEE,Capital_Lit_MCO,FI_EJ,SALTOT
0,010008407,1.650548e+05,1,01,01283,102.0,010008407,7.0
1,010780054,7.879291e+05,1,01,01451,369.0,010780054,12.0
2,010780062,1.881749e+05,1,01,01034,93.0,010780062,4.0
3,020000063,8.671006e+05,1,02,02691,395.0,020000063,19.0
4,020000253,3.218348e+05,1,02,02408,183.0,020000253,6.0
...,...,...,...,...,...,...,...,...
451,970403606,3.325696e+05,1,9D,9D410,177.0,970403606,6.0
452,970408589,2.697187e+06,1,9D,9D411,245.6,970408589,19.0
453,970408589,2.697187e+06,1,9D,9D411,245.6,970408589,26.0
454,970421038,5.785114e+05,1,9D,9D415,276.0,970421038,7.0


In [34]:
bloc_fi_ej_salto.FI_EJ.nunique()

357

In [35]:
# Affichage des duplicatas 
bloc_fi_ej_salto[bloc_fi_ej_salto.duplicated(subset ='Numero_finess')]

,Numero_finess,Points_ISA,stjr,dep,COMINSEE,Capital_Lit_MCO,FI_EJ,SALTOT
15,050002948,6.212138e+05,1,04,04209,140.000000,050002948,7.0
20,060785011,2.515840e+06,1,06,06088,266.500000,060785011,29.0
21,060785011,2.515840e+06,1,06,06088,266.500000,060785011,15.0
30,090781774,4.851891e+05,1,09,09122,121.000000,090781774,0.0
42,130041916,1.065176e+06,1,13,13001,252.000000,130041916,2.0
...,...,...,...,...,...,...,...,...
443,970100228,9.136090e+05,1,9A,9A101,174.333333,970100228,16.0
445,970211207,1.597206e+06,1,9B,9B209,153.600000,970211207,3.0
446,970211207,1.597206e+06,1,9B,9B209,153.600000,970211207,6.0
447,970211207,1.597206e+06,1,9B,9B209,153.600000,970211207,6.0


In [36]:
# On effectue la moyenne :
df_fiunique_bloc_salto = pd.DataFrame({'Numero_finess':bloc_fi_ej_salto.groupby('FI_EJ', as_index=False).mean()['FI_EJ'], 'FI_EJ':bloc_fi_ej_salto.groupby('FI_EJ', as_index=False).mean()['FI_EJ'], 'stjr':bloc_fi_ej_salto.groupby('FI_EJ', as_index=False).mean()['stjr'],'SALTOT': bloc_fi_ej_salto.groupby('FI_EJ', as_index=False).mean()['SALTOT'] })
# On retire les doublons 
bloc_fi_ej_salto.drop_duplicates(subset ='Numero_finess', keep = 'first', inplace=True)
# On fusionne
bloc_fiej_final_salto = pd.merge (df_fiunique_bloc_salto,bloc_fi_ej_salto[['Numero_finess','Points_ISA','dep','COMINSEE','Capital_Lit_MCO']], left_on='Numero_finess', right_on='Numero_finess',how='inner')
bloc_fiej_final_salto

,Numero_finess,FI_EJ,stjr,SALTOT,Points_ISA,dep,COMINSEE,Capital_Lit_MCO
0,010008407,010008407,1,7.0,1.650548e+05,01,01283,102.0
1,010780054,010780054,1,12.0,7.879291e+05,01,01451,369.0
2,010780062,010780062,1,4.0,1.881749e+05,01,01034,93.0
3,020000063,020000063,1,19.0,8.671006e+05,02,02691,395.0
4,020000253,020000253,1,6.0,3.218348e+05,02,02408,183.0
...,...,...,...,...,...,...,...,...
352,970305629,970305629,1,4.0,9.421212e+04,9C,9C304,92.0
353,970403606,970403606,1,6.0,3.325696e+05,9D,9D410,177.0
354,970408589,970408589,1,22.5,2.697187e+06,9D,9D411,245.6
355,970421038,970421038,1,7.0,5.785114e+05,9D,9D415,276.0


In [37]:
bloc_salto =pd.concat([bloc_fi_salto, bloc_fiej_final_salto])
bloc_salto

,Numero_finess,Points_ISA,stjr,dep,COMINSEE,Capital_Lit_MCO,FI,SALTOT,FI_EJ
0,060000528,3.568920e+05,3,06,06088,128.0,060000528,8.0,NaN
1,060780491,3.724104e+05,3,06,06123,128.0,060780491,14.0,NaN
2,060780947,1.745525e+05,3,06,06088,105.0,060780947,5.0,NaN
3,060791811,1.464326e+05,3,06,06088,91.0,060791811,2.0,NaN
4,060794013,2.381733e+05,3,06,06123,68.0,060794013,5.0,NaN
...,...,...,...,...,...,...,...,...,...
352,970305629,9.421212e+04,1,9C,9C304,92.0,NaN,4.0,970305629
353,970403606,3.325696e+05,1,9D,9D410,177.0,NaN,6.0,970403606
354,970408589,2.697187e+06,1,9D,9D411,245.6,NaN,22.5,970408589
355,970421038,5.785114e+05,1,9D,9D415,276.0,NaN,7.0,970421038


Il y a 272 établissements manquants :

In [38]:
list1 = bloc_salto.Numero_finess.unique()
list2 = Var_CTRL.Numero_finess.unique()
res_bloc = [value for value in list2 if value not in list1]
res_bloc

['130045263',
 '130781255',
 '130783152',
 '130784226',
 '130811102',
 '140000340',
 '220000236',
 '290000975',
 '330000217',
 '330780370',
 '330781139',
 '340000025',
 '340781608',
 '350000063',
 '350000071',
 '350000204',
 '350002200',
 '350005021',
 '400000261',
 '420000192',
 '450000336',
 '490000700',
 '490004256',
 '540000072',
 '540000395',
 '540001104',
 '560002974',
 '570000026',
 '570000166',
 '570000455',
 '570000562',
 '570001099',
 '570003079',
 '570009670',
 '590049565',
 '620003350',
 '660009689',
 '670014042',
 '670017458',
 '670797539',
 '680000882',
 '690000245',
 '740780192',
 '750055287',
 '750150146',
 '750150187',
 '750150286',
 '750150344',
 '750150377',
 '780150033',
 '780150066',
 '780630026',
 '810000448',
 '910150010',
 '910150028',
 '910150069',
 '910500040',
 '920000635',
 '920300845',
 '940000656',
 '950000406',
 '950630012',
 '130002694',
 '130789159',
 '150780120',
 '280000449',
 '2B0005664',
 '310021571',
 '310781695',
 '330780354',
 '360002232',
 '4300

Les établissements absents de la base Bloc n'ont pas de bloc opératoire, c'est pourquoi ils ne sont pas présents dans la base BLOC de la SAE. Nous les rajoutons en établissant à 0 le nombre de salle de bloc opératoire. 

In [39]:
fi_ss_bloc = Var_CTRL.loc[Var_CTRL.Numero_finess.isin(res_bloc)]
# On établie à 0 le nombre de salle : 
fi_ss_bloc['SALTOT'] = 0
fi_ss_bloc

<ipython-input-39-8d195be36f22>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fi_ss_bloc['SALTOT'] = 0


,Numero_finess,Points_ISA,stjr,dep,COMINSEE,Capital_Lit_MCO,SALTOT
12,130045263,5098.133333,3,13,13041,14.0,0
13,130781255,19287.983333,3,13,13001,40.0,0
14,130783152,5850.013333,3,13,13055,12.0,0
16,130784226,3828.400000,3,13,13055,0.0,0
20,130811102,5098.133333,3,13,13041,24.0,0
...,...,...,...,...,...,...,...
497,920110053,62641.130000,1,92,92063,48.0,0
507,950015289,18313.888333,1,95,95355,26.0,0
511,970100160,3433.260000,1,9A,9A123,10.0,0
514,970100202,5081.250000,1,9A,9A112,17.0,0


In [40]:
Var_CTRL = pd.concat([bloc_salto,fi_ss_bloc]) 
Var_CTRL

,Numero_finess,Points_ISA,stjr,dep,COMINSEE,Capital_Lit_MCO,FI,SALTOT,FI_EJ
0,060000528,356892.030000,3,06,06088,128.0,060000528,8.0,NaN
1,060780491,372410.435000,3,06,06123,128.0,060780491,14.0,NaN
2,060780947,174552.540000,3,06,06088,105.0,060780947,5.0,NaN
3,060791811,146432.633333,3,06,06088,91.0,060791811,2.0,NaN
4,060794013,238173.260000,3,06,06123,68.0,060794013,5.0,NaN
...,...,...,...,...,...,...,...,...,...
497,920110053,62641.130000,1,92,92063,48.0,NaN,0.0,NaN
507,950015289,18313.888333,1,95,95355,26.0,NaN,0.0,NaN
511,970100160,3433.260000,1,9A,9A123,10.0,NaN,0.0,NaN
514,970100202,5081.250000,1,9A,9A112,17.0,NaN,0.0,NaN


In [41]:
Var_CTRL= Var_CTRL.drop(['FI','FI_EJ'], axis=1)

In [42]:
Var_CTRL

,Numero_finess,Points_ISA,stjr,dep,COMINSEE,Capital_Lit_MCO,SALTOT
0,060000528,356892.030000,3,06,06088,128.0,8.0
1,060780491,372410.435000,3,06,06123,128.0,14.0
2,060780947,174552.540000,3,06,06088,105.0,5.0
3,060791811,146432.633333,3,06,06088,91.0,2.0
4,060794013,238173.260000,3,06,06123,68.0,5.0
...,...,...,...,...,...,...,...
497,920110053,62641.130000,1,92,92063,48.0,0.0
507,950015289,18313.888333,1,95,95355,26.0,0.0
511,970100160,3433.260000,1,9A,9A123,10.0,0.0
514,970100202,5081.250000,1,9A,9A112,17.0,0.0


In [43]:
Var_CTRL= Var_CTRL.rename(columns={'SALTOT' : 'Capital_Bloc'})

#### IMAGES 

On cherche un indicateur du capital corporel : les équipements de l'imagerie (lourds et conventionnels). La variable NB représente le nombre d'appareils présents sur le site en fonction de l'équipement ( IRM, Scanner...) 

In [44]:
IMAGES

,BOR,AN,FI,FI_EJ,EQUIPIMA,NB,NBEXP,ACTET,ACTEXET,ACTEXEJ,ACTEX,ACTNUI,ACTINT
0,IMAGES,2019,010000024,010780054,IRM,2.0,2.0,2087.0,29.0,NaN,6838.0,12.0,NaN
1,IMAGES,2019,010000024,010780054,MAMM,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,IMAGES,2019,010000024,010780054,RADIO,4.0,4.0,17162.0,390.0,NaN,25015.0,461.0,1.0
3,IMAGES,2019,010000024,010780054,SCAN,2.0,2.0,7539.0,160.0,NaN,8285.0,236.0,1.0
4,IMAGES,2019,010000024,010780054,VASC,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3539,IMAGES,2019,970463139,970408589,RADIO,1.0,1.0,214.0,NaN,NaN,NaN,NaN,0.0
3540,IMAGES,2019,980500011,980500003,IRM,1.0,1.0,646.0,NaN,NaN,1192.0,3.0,0.0
3541,IMAGES,2019,980500011,980500003,MAMM,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
3542,IMAGES,2019,980500011,980500003,RADIO,4.0,4.0,6429.0,NaN,NaN,29404.0,1102.0,0.0


In [45]:
IMAGES[['FI','FI_EJ','NBEXP', 'EQUIPIMA']]

,FI,FI_EJ,NBEXP,EQUIPIMA
0,010000024,010780054,2.0,IRM
1,010000024,010780054,NaN,MAMM
2,010000024,010780054,4.0,RADIO
3,010000024,010780054,2.0,SCAN
4,010000024,010780054,NaN,VASC
...,...,...,...,...
3539,970463139,970408589,1.0,RADIO
3540,980500011,980500003,1.0,IRM
3541,980500011,980500003,1.0,MAMM
3542,980500011,980500003,4.0,RADIO


In [46]:
Images_fi = pd.merge (Var_CTRL, IMAGES[['FI','NB','EQUIPIMA']], left_on='Numero_finess', right_on='FI',how='inner')
Images_fi

,Numero_finess,Points_ISA,stjr,dep,COMINSEE,Capital_Lit_MCO,Capital_Bloc,FI,NB,EQUIPIMA
0,060000528,356892.030000,3,06,06088,128.0,8.0,060000528,3.0,CAM
1,060000528,356892.030000,3,06,06088,128.0,8.0,060000528,1.0,IRM
2,060000528,356892.030000,3,06,06088,128.0,8.0,060000528,2.0,MAMM
3,060000528,356892.030000,3,06,06088,128.0,8.0,060000528,1.0,RADIO
4,060000528,356892.030000,3,06,06088,128.0,8.0,060000528,1.0,SCAN
...,...,...,...,...,...,...,...,...,...,...
1239,920300266,19690.966667,2,92,92023,24.0,0.0,920300266,3.0,RADIO
1240,940300163,4440.630000,2,94,94038,20.0,0.0,940300163,1.0,RADIO
1241,940300452,11731.190000,2,94,94075,40.0,0.0,940300452,1.0,RADIO
1242,970102596,17963.580000,2,9A,9A113,27.0,0.0,970102596,1.0,RADIO


In [47]:
nb_imagerie_fi = Images_fi[['Numero_finess','NB']].groupby('Numero_finess').sum()
nb_imagerie_fi

,NB
Numero_finess,
010780195,7.0
020000360,3.0
030780548,4.0
030781116,1.0
060000528,9.0
...,...
970302055,3.0
970404844,5.0
970462024,2.0


In [48]:
Images_fi.FI.nunique()

360

In [49]:
Images_fi_ej = pd.merge (Var_CTRL, IMAGES[['FI_EJ','NB','EQUIPIMA']], left_on='Numero_finess', right_on='FI_EJ',how='inner')
Images_fi_ej

,Numero_finess,Points_ISA,stjr,dep,COMINSEE,Capital_Lit_MCO,Capital_Bloc,FI_EJ,NB,EQUIPIMA
0,010008407,165054.798333,1,01,01283,102.0,7.0,010008407,1.0,IRM
1,010008407,165054.798333,1,01,01283,102.0,7.0,010008407,3.0,RADIO
2,010008407,165054.798333,1,01,01283,102.0,7.0,010008407,1.0,SCAN
3,010780054,787929.068333,1,01,01451,369.0,12.0,010780054,2.0,IRM
4,010780054,787929.068333,1,01,01451,369.0,12.0,010780054,1.0,MAMM
...,...,...,...,...,...,...,...,...,...,...
2104,950015289,18313.888333,1,95,95355,26.0,0.0,950015289,1.0,RADIO
2105,970100202,5081.250000,1,9A,9A112,17.0,0.0,970100202,1.0,MAMM
2106,970100202,5081.250000,1,9A,9A112,17.0,0.0,970100202,1.0,RADIO
2107,970100202,5081.250000,1,9A,9A112,17.0,0.0,970100202,1.0,SCAN


In [50]:
nb_par_groupe_fi_ej = Images_fi_ej[['Numero_finess','NB', 'EQUIPIMA']].groupby(['Numero_finess', 'EQUIPIMA']).mean()
nb_imagerie_fi_ej = nb_par_groupe_fi_ej.groupby('Numero_finess').sum()

In [51]:
nb_imagerie= pd.concat([nb_imagerie_fi, nb_imagerie_fi_ej])
nb_imagerie

,NB
Numero_finess,
010780195,7.0
020000360,3.0
030780548,4.0
030781116,1.0
060000528,9.0
...,...
970302121,5.0
970305629,6.0
970408589,13.8


In [52]:
Var_CTRL_Im = pd.merge(Var_CTRL,nb_imagerie, left_on='Numero_finess', right_on=nb_imagerie.index ,how='inner' )
Var_CTRL_Im

,Numero_finess,Points_ISA,stjr,dep,COMINSEE,Capital_Lit_MCO,Capital_Bloc,NB
0,060000528,356892.030000,3,06,06088,128.0,8.0,9.0
1,060780491,372410.435000,3,06,06123,128.0,14.0,10.0
2,060780947,174552.540000,3,06,06088,105.0,5.0,4.0
3,060791811,146432.633333,3,06,06088,91.0,2.0,2.0
4,060794013,238173.260000,3,06,06123,68.0,5.0,7.0
...,...,...,...,...,...,...,...,...
778,890000417,92778.473333,1,89,89206,72.0,0.0,4.0
779,920110053,62641.130000,1,92,92063,48.0,0.0,3.0
780,950015289,18313.888333,1,95,95355,26.0,0.0,2.5
781,970100202,5081.250000,1,9A,9A112,17.0,0.0,3.0


In [53]:
Var_CTRL

,Numero_finess,Points_ISA,stjr,dep,COMINSEE,Capital_Lit_MCO,Capital_Bloc
0,060000528,356892.030000,3,06,06088,128.0,8.0
1,060780491,372410.435000,3,06,06123,128.0,14.0
2,060780947,174552.540000,3,06,06088,105.0,5.0
3,060791811,146432.633333,3,06,06088,91.0,2.0
4,060794013,238173.260000,3,06,06123,68.0,5.0
...,...,...,...,...,...,...,...
497,920110053,62641.130000,1,92,92063,48.0,0.0
507,950015289,18313.888333,1,95,95355,26.0,0.0
511,970100160,3433.260000,1,9A,9A123,10.0,0.0
514,970100202,5081.250000,1,9A,9A112,17.0,0.0


In [54]:
list2= Var_CTRL.Numero_finess.unique()
list1 = Var_CTRL_Im.Numero_finess.unique()
res = [value for value in list2 if value not in list1]
res

['080010465',
 '080010473',
 '250011848',
 '330060658',
 '350000139',
 '370013286',
 '380012658',
 '390780575',
 '420010050',
 '470016049',
 '520004680',
 '520004714',
 '590052056',
 '590785374',
 '600100168',
 '640016580',
 '660006305',
 '670020098',
 '670780212',
 '680000320',
 '680020088',
 '690000427',
 '690041132',
 '690780416',
 '730012499',
 '750150013',
 '750150260',
 '750300667',
 '770020477',
 '920000643',
 '930150032',
 '010007300',
 '010780203',
 '020010047',
 '030785430',
 '040780470',
 '050000090',
 '060021417',
 '060780517',
 '060785219',
 '060800166',
 '070780168',
 '100000082',
 '100000157',
 '110780483',
 '130008253',
 '130044753',
 '130781289',
 '130781370',
 '130781867',
 '140000290',
 '140016759',
 '140018730',
 '140026709',
 '160000279',
 '160013207',
 '170780613',
 '170780647',
 '170780662',
 '180004145',
 '190000257',
 '210011847',
 '220005599',
 '220022800',
 '230780157',
 '240000190',
 '240000208',
 '240000216',
 '260000260',
 '280505777',
 '290004142',
 '2B00

Ce ne sont pas exactement les memes établissements qui n'ont pas de bloc opératoire et pas d'équipement d'imagerie. 

In [55]:
fi_ss_imagerie = Var_CTRL.loc[Var_CTRL.Numero_finess.isin(res)]
# On établie à 0 le nombre de salle : 
fi_ss_imagerie['NB'] = 0
fi_ss_imagerie

<ipython-input-55-d58d98cf81ce>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fi_ss_imagerie['NB'] = 0


,Numero_finess,Points_ISA,stjr,dep,COMINSEE,Capital_Lit_MCO,Capital_Bloc,NB
6,080010465,4938.370000,3,08,08409,0.000000,2.0,0
7,080010473,118109.733333,3,08,08105,76.000000,7.0,0
20,250011848,233597.895000,3,25,25056,103.000000,13.0,0
29,330060658,65560.425000,3,33,33243,35.000000,7.0,0
36,350000139,291201.275000,3,35,35238,143.000000,15.0,0
...,...,...,...,...,...,...,...,...
459,840000079,5098.133333,1,84,84054,8.000000,0.0,0
477,870014503,22991.543333,1,87,87089,15.666667,0.0,0
483,880780325,2339.030000,1,88,88181,8.000000,0.0,0
487,890000433,39747.000000,1,89,89418,50.000000,0.0,0


In [56]:
Var_CTRL = pd.concat([Var_CTRL_Im,fi_ss_imagerie])
Var_CTRL= Var_CTRL.rename(columns={'NB' : 'Capital_imagerie'})

In [57]:
Var_CTRL

,Numero_finess,Points_ISA,stjr,dep,COMINSEE,Capital_Lit_MCO,Capital_Bloc,Capital_imagerie
0,060000528,356892.030000,3,06,06088,128.000000,8.0,9.0
1,060780491,372410.435000,3,06,06123,128.000000,14.0,10.0
2,060780947,174552.540000,3,06,06088,105.000000,5.0,4.0
3,060791811,146432.633333,3,06,06088,91.000000,2.0,2.0
4,060794013,238173.260000,3,06,06123,68.000000,5.0,7.0
...,...,...,...,...,...,...,...,...
459,840000079,5098.133333,1,84,84054,8.000000,0.0,0.0
477,870014503,22991.543333,1,87,87089,15.666667,0.0,0.0
483,880780325,2339.030000,1,88,88181,8.000000,0.0,0.0
487,890000433,39747.000000,1,89,89418,50.000000,0.0,0.0


#### BIO

La variable BIO_B2 est l'activité produite pour des patients pris en charge par l'entité juridique (y compris les consultations externes et passages aux urgences sans hospitalisation), en nombre d'examens. Cela permet d'avoir une idée des ressources engagés dans les examens biologiques

In [58]:
BIO[['FI','FI_EJ','BIO_B2']]

,FI,FI_EJ,BIO_B2
0,010000024,010780054,954696.0
1,010005239,010008407,265742.0
2,020000162,020000063,1492636.0
3,020000394,020000253,984982.0
4,020000519,020000261,771938.0
...,...,...,...
396,970300083,970302121,102225.0
397,970400024,970408589,1434783.0
398,970400057,970408589,1765696.0
399,970400065,970421038,895633.0


In [59]:
bio_fi = pd.merge(Var_CTRL,BIO[['FI','BIO_B2']], left_on='Numero_finess', right_on='FI',how='inner'  )
bio_fi_ej = pd.merge(Var_CTRL,BIO[['FI_EJ','BIO_B2']], left_on='Numero_finess', right_on='FI_EJ',how='inner'  )

In [60]:
bio_fi_ej

,Numero_finess,Points_ISA,stjr,dep,COMINSEE,Capital_Lit_MCO,Capital_Bloc,Capital_imagerie,FI_EJ,BIO_B2
0,010008407,165054.798333,1,01,01283,102.0,7.0,5.0,010008407,265742.0
1,010780054,787929.068333,1,01,01451,369.0,12.0,10.0,010780054,954696.0
2,020000063,867100.583333,1,02,02691,395.0,19.0,12.0,020000063,1492636.0
3,020000253,321834.750000,1,02,02408,183.0,6.0,6.0,020000253,984982.0
4,020000261,533474.643333,1,02,02722,271.0,8.0,9.0,020000261,771938.0
...,...,...,...,...,...,...,...,...,...,...
328,850000035,158046.038333,1,85,85092,124.0,2.0,0.0,850000035,240534.0
329,920009909,358944.708333,1,92,92064,100.5,5.0,0.0,920009909,602099.0
330,450000096,139867.888333,1,45,45155,114.0,0.0,0.0,450000096,197398.0
331,520780057,88997.565000,1,52,52269,71.0,0.0,0.0,520780057,204872.0


In [61]:
# On fait la moyenne pour les doublons de FI_EJ : 
moy_fi_ej = bio_fi_ej[['Numero_finess','BIO_B2']].groupby('Numero_finess').mean()
# On supprimer dans la base
bio_fi_ej.drop_duplicates(subset ='Numero_finess', keep = 'first', inplace=True)
bio_fi_ej= bio_fi_ej.drop(['BIO_B2'], axis=1)
# On ajoute les bonne valeur 
bio_fi_ej_final = pd.merge(bio_fi_ej,moy_fi_ej, left_on='Numero_finess', right_on=moy_fi_ej.index)

In [62]:
# On concat les fi et fi_ej 
Var_CTRL_bio = pd.concat([bio_fi,bio_fi_ej_final])

In [63]:
Var_CTRL_bio

,Numero_finess,Points_ISA,stjr,dep,COMINSEE,Capital_Lit_MCO,Capital_Bloc,Capital_imagerie,FI,BIO_B2,FI_EJ
0,060000528,3.568920e+05,3,06,06088,128.0,8.0,9.0,060000528,909.0,NaN
1,130001647,8.547880e+05,3,13,13055,271.0,12.0,13.0,130001647,16870.0,NaN
2,130783665,6.971615e+04,3,13,13055,49.0,5.0,6.0,130783665,71043.0,NaN
3,130785652,1.618811e+06,3,13,13055,678.0,29.0,14.0,130785652,341422.0,NaN
4,130786742,2.939701e+05,1,13,13055,132.0,8.0,8.0,130786742,596640.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
262,850000035,1.580460e+05,1,85,85092,124.0,2.0,0.0,NaN,240534.0,850000035
263,920009909,3.589447e+05,1,92,92064,100.5,5.0,0.0,NaN,602099.0,920009909
264,450000096,1.398679e+05,1,45,45155,114.0,0.0,0.0,NaN,197398.0,450000096
265,520780057,8.899756e+04,1,52,52269,71.0,0.0,0.0,NaN,204872.0,520780057


In [64]:
liste1 = Var_CTRL_bio.Numero_finess.unique()
liste2 = Var_CTRL.Numero_finess.unique()
res_bio = [value for value in liste2 if value not in liste1]
res_bio

['060780491',
 '060780947',
 '060791811',
 '060794013',
 '070000096',
 '100010818',
 '130042096',
 '130043664',
 '130786445',
 '140002452',
 '230780082',
 '260000195',
 '310781067',
 '330000332',
 '330000340',
 '330780495',
 '330780529',
 '330780537',
 '340780642',
 '350002192',
 '380780197',
 '440029338',
 '440050433',
 '540001096',
 '560000184',
 '560002933',
 '560029068',
 '570000216',
 '570000596',
 '570000950',
 '570001057',
 '590001749',
 '620000026',
 '620003376',
 '620025346',
 '640780557',
 '670000033',
 '670000215',
 '670018068',
 '670780188',
 '670780378',
 '670798636',
 '680001195',
 '690780093',
 '690793468',
 '710978347',
 '720000389',
 '750150237',
 '750150252',
 '750300097',
 '750300121',
 '830200523',
 '840000350',
 '010780195',
 '020000360',
 '030780548',
 '030781116',
 '060780442',
 '060780590',
 '060780715',
 '060780723',
 '060780756',
 '060781200',
 '070780424',
 '100000124',
 '110780228',
 '130037922',
 '130041767',
 '130782071',
 '130782162',
 '130782675',
 '1307

In [65]:
fi_ss_bio = Var_CTRL.loc[Var_CTRL.Numero_finess.isin(res_bio)]
# On établie à 0 le nombre de salle : 
fi_ss_bio['BIO_B2'] = 0
fi_ss_bio

<ipython-input-65-a399fbe181b8>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fi_ss_bio['BIO_B2'] = 0


,Numero_finess,Points_ISA,stjr,dep,COMINSEE,Capital_Lit_MCO,Capital_Bloc,Capital_imagerie,BIO_B2
1,060780491,372410.435000,3,06,06123,128.000000,14.0,10.0,0
2,060780947,174552.540000,3,06,06088,105.000000,5.0,4.0,0
3,060791811,146432.633333,3,06,06088,91.000000,2.0,2.0,0
4,060794013,238173.260000,3,06,06123,68.000000,5.0,7.0,0
5,070000096,1562.320000,3,07,07204,8.000000,1.0,1.0,0
...,...,...,...,...,...,...,...,...,...
447,810000398,5098.133333,1,81,81105,18.000000,0.0,0.0,0
459,840000079,5098.133333,1,84,84054,8.000000,0.0,0.0,0
477,870014503,22991.543333,1,87,87089,15.666667,0.0,0.0,0
483,880780325,2339.030000,1,88,88181,8.000000,0.0,0.0,0


In [66]:
Var_CTRL = pd.concat([Var_CTRL_bio,fi_ss_bio])
Var_CTRL

,Numero_finess,Points_ISA,stjr,dep,COMINSEE,Capital_Lit_MCO,Capital_Bloc,Capital_imagerie,FI,BIO_B2,FI_EJ
0,060000528,3.568920e+05,3,06,06088,128.000000,8.0,9.0,060000528,909.0,NaN
1,130001647,8.547880e+05,3,13,13055,271.000000,12.0,13.0,130001647,16870.0,NaN
2,130783665,6.971615e+04,3,13,13055,49.000000,5.0,6.0,130783665,71043.0,NaN
3,130785652,1.618811e+06,3,13,13055,678.000000,29.0,14.0,130785652,341422.0,NaN
4,130786742,2.939701e+05,1,13,13055,132.000000,8.0,8.0,130786742,596640.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
447,810000398,5.098133e+03,1,81,81105,18.000000,0.0,0.0,NaN,0.0,NaN
459,840000079,5.098133e+03,1,84,84054,8.000000,0.0,0.0,NaN,0.0,NaN
477,870014503,2.299154e+04,1,87,87089,15.666667,0.0,0.0,NaN,0.0,NaN
483,880780325,2.339030e+03,1,88,88181,8.000000,0.0,0.0,NaN,0.0,NaN


In [67]:
Var_CTRL = pd.concat([Var_CTRL_bio,fi_ss_bio])
Var_CTRL= Var_CTRL.rename(columns={'BIO_B2' : 'Capital_Examens_biologiques'})
Var_CTRL= Var_CTRL.drop(['FI','FI_EJ'], axis=1)

#### TELEMED 

In [68]:
TELEMED

,BOR,AN,FI,FI_EJ,TELEMED_A1,TELEMED_A20,TELEMED_A21,TELEMED_A22,TELEMED_B1,TELEMED_B20,...,TELEMED_A29,TELEMED_A30,TELEMED_A31,TELEMED_A32,TELEMED_A33,TELEMED_A34,TELEMED_A35,TELEMED_A36,TELEMED_A37,TELEMED_A38
0,TELEMED,2019,010000024,010780054,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,TELEMED,2019,010000081,010780112,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,TELEMED,2019,010000214,010007987,0.0,NaN,NaN,NaN,0.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,TELEMED,2019,010000495,010783009,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,TELEMED,2019,010006526,690796552,1.0,1.0,0.0,0.0,0.0,NaN,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
825,TELEMED,2019,970405676,970463592,1.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
826,TELEMED,2019,970407151,970463592,1.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
827,TELEMED,2019,970409991,970463592,1.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
828,TELEMED,2019,970410163,970463592,1.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


On se concentre tout d'abord sur les équipements dédiés à la télémedecine. Ils sont représentés par les variable TELEMED_A26 jusque TELEMED_A38 

In [69]:
TELEMED[['FI','FI_EJ','TELEMED_A26','TELEMED_A27','TELEMED_A28','TELEMED_A29','TELEMED_A30','TELEMED_A31','TELEMED_A32','TELEMED_A33','TELEMED_A34','TELEMED_A35','TELEMED_A36','TELEMED_A37','TELEMED_A38']]

,FI,FI_EJ,TELEMED_A26,TELEMED_A27,TELEMED_A28,TELEMED_A29,TELEMED_A30,TELEMED_A31,TELEMED_A32,TELEMED_A33,TELEMED_A34,TELEMED_A35,TELEMED_A36,TELEMED_A37,TELEMED_A38
0,010000024,010780054,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,010000081,010780112,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,010000214,010007987,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,010000495,010783009,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,010006526,690796552,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
825,970405676,970463592,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
826,970407151,970463592,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
827,970409991,970463592,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
828,970410163,970463592,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [70]:
TELEMED['Capital_telemed']=TELEMED[['TELEMED_A26','TELEMED_A27','TELEMED_A28','TELEMED_A29','TELEMED_A30','TELEMED_A31','TELEMED_A32','TELEMED_A33','TELEMED_A34','TELEMED_A35','TELEMED_A36','TELEMED_A37','TELEMED_A38']].sum(axis=1)

In [71]:
TELEMED_1 = TELEMED[['FI','FI_EJ','Capital_telemed']]
TELEMED_1

,FI,FI_EJ,Capital_telemed
0,010000024,010780054,2.0
1,010000081,010780112,2.0
2,010000214,010007987,0.0
3,010000495,010783009,2.0
4,010006526,690796552,1.0
...,...,...,...
825,970405676,970463592,2.0
826,970407151,970463592,2.0
827,970409991,970463592,2.0
828,970410163,970463592,2.0


In [72]:
TELEMED_1_fi = pd.merge(Var_CTRL,TELEMED_1,left_on='Numero_finess', right_on='FI',how='inner' )
TELEMED_1_fi

,Numero_finess,Points_ISA,stjr,dep,COMINSEE,Capital_Lit_MCO,Capital_Bloc,Capital_imagerie,Capital_Examens_biologiques,FI,FI_EJ,Capital_telemed
0,060000528,356892.030000,3,06,06088,128.0,8.0,9.0,909.0,060000528,060780962,1.0
1,130001647,854787.985000,3,13,13055,271.0,12.0,13.0,16870.0,130001647,130784127,0.0
2,210987731,205661.893333,3,21,21231,131.0,8.0,11.0,280257.0,210987731,210780417,0.0
3,310782347,170147.100000,3,31,31555,94.0,10.0,8.0,435987.0,310782347,310789136,1.0
4,350002812,61024.268333,3,35,35238,54.0,4.0,9.0,140425.0,350002812,350023503,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...
100,350000204,8416.853333,3,35,35085,15.0,0.0,0.0,0.0,350000204,350023248,2.0
101,750150146,82230.863333,3,75,75056,50.0,0.0,0.0,0.0,750150146,750720609,3.0
102,310021571,3751.130000,2,31,31555,14.0,0.0,0.0,0.0,310021571,310021563,1.0
103,360002232,11685.336667,2,36,36163,25.0,0.0,0.0,0.0,360002232,360000541,11.0


In [73]:
TELEMED_1_fi_ej = pd.merge(Var_CTRL,TELEMED_1,left_on='Numero_finess', right_on='FI_EJ',how='inner' )
TELEMED_1_fi_ej

,Numero_finess,Points_ISA,stjr,dep,COMINSEE,Capital_Lit_MCO,Capital_Bloc,Capital_imagerie,Capital_Examens_biologiques,FI,FI_EJ,Capital_telemed
0,010780054,787929.068333,1,01,01451,369.000000,12.0,10.0,954696.0,010000024,010780054,2.0
1,020000063,867100.583333,1,02,02691,395.000000,19.0,12.0,1492636.0,020000162,020000063,4.0
2,020000261,533474.643333,1,02,02722,271.000000,8.0,9.0,771938.0,020000519,020000261,0.0
3,020004404,373304.290000,1,02,02168,168.000000,6.0,6.0,594902.0,020001061,020004404,1.0
4,030780092,554335.088333,1,03,03190,311.000000,7.0,9.0,714739.0,030000061,030780092,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...
427,710781592,5098.133333,1,71,71073,20.000000,0.0,0.0,0.0,710978107,710781592,1.0
428,810000398,5098.133333,1,81,81105,18.000000,0.0,0.0,0.0,810000539,810000398,1.0
429,870014503,22991.543333,1,87,87089,15.666667,0.0,0.0,0.0,870001567,870014503,2.0
430,880780325,2339.030000,1,88,88181,8.000000,0.0,0.0,0.0,880000179,880780325,7.0


In [74]:
# On fait la moyenne pour les doublons de FI_EJ : 
moy_fi_ej = TELEMED_1_fi_ej[['Numero_finess','Capital_telemed']].groupby('Numero_finess').mean()
# On supprimer dans la base
TELEMED_1_fi_ej.drop_duplicates(subset ='Numero_finess', keep = 'first', inplace=True)
TELEMED_1_fi_ej= TELEMED_1_fi_ej.drop(['Capital_telemed'], axis=1)
# On ajoute les bonne valeur 
TELEMED_1_fi_ej_final = pd.merge(TELEMED_1_fi_ej,moy_fi_ej, left_on='Numero_finess', right_on=moy_fi_ej.index)

In [75]:
TELEMED_1_fi_ej_final

,Numero_finess,Points_ISA,stjr,dep,COMINSEE,Capital_Lit_MCO,Capital_Bloc,Capital_imagerie,Capital_Examens_biologiques,FI,FI_EJ,Capital_telemed
0,010780054,787929.068333,1,01,01451,369.000000,12.0,10.0,954696.0,010000024,010780054,2.0
1,020000063,867100.583333,1,02,02691,395.000000,19.0,12.0,1492636.0,020000162,020000063,4.0
2,020000261,533474.643333,1,02,02722,271.000000,8.0,9.0,771938.0,020000519,020000261,0.0
3,020004404,373304.290000,1,02,02168,168.000000,6.0,6.0,594902.0,020001061,020004404,1.0
4,030780092,554335.088333,1,03,03190,311.000000,7.0,9.0,714739.0,030000061,030780092,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...
317,710781592,5098.133333,1,71,71073,20.000000,0.0,0.0,0.0,710978107,710781592,1.0
318,810000398,5098.133333,1,81,81105,18.000000,0.0,0.0,0.0,810000539,810000398,1.0
319,870014503,22991.543333,1,87,87089,15.666667,0.0,0.0,0.0,870001567,870014503,2.0
320,880780325,2339.030000,1,88,88181,8.000000,0.0,0.0,0.0,880000179,880780325,7.0


In [76]:
# On concat les fi et fi_ej 
Var_CTRL_TELEMED_1= pd.concat([TELEMED_1_fi,TELEMED_1_fi_ej_final])
Var_CTRL_TELEMED_1

,Numero_finess,Points_ISA,stjr,dep,COMINSEE,Capital_Lit_MCO,Capital_Bloc,Capital_imagerie,Capital_Examens_biologiques,FI,FI_EJ,Capital_telemed
0,060000528,356892.030000,3,06,06088,128.000000,8.0,9.0,909.0,060000528,060780962,1.0
1,130001647,854787.985000,3,13,13055,271.000000,12.0,13.0,16870.0,130001647,130784127,0.0
2,210987731,205661.893333,3,21,21231,131.000000,8.0,11.0,280257.0,210987731,210780417,0.0
3,310782347,170147.100000,3,31,31555,94.000000,10.0,8.0,435987.0,310782347,310789136,1.0
4,350002812,61024.268333,3,35,35238,54.000000,4.0,9.0,140425.0,350002812,350023503,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...
317,710781592,5098.133333,1,71,71073,20.000000,0.0,0.0,0.0,710978107,710781592,1.0
318,810000398,5098.133333,1,81,81105,18.000000,0.0,0.0,0.0,810000539,810000398,1.0
319,870014503,22991.543333,1,87,87089,15.666667,0.0,0.0,0.0,870001567,870014503,2.0
320,880780325,2339.030000,1,88,88181,8.000000,0.0,0.0,0.0,880000179,880780325,7.0


In [77]:
# Identification des Finess manquants
list2 = Var_CTRL.Numero_finess.unique()
list1 = Var_CTRL_TELEMED_1.Numero_finess.unique()
res_telemed_1 = [value for value in list2 if value not in list1]

In [78]:
fi_ss_telemed_1 = Var_CTRL.loc[Var_CTRL.Numero_finess.isin(res_telemed_1)]
# On établie à 0 le nombre de salle : 
fi_ss_telemed_1['Capital_telemed'] = 0
fi_ss_telemed_1

<ipython-input-78-b654f2cda5f9>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fi_ss_telemed_1['Capital_telemed'] = 0


,Numero_finess,Points_ISA,stjr,dep,COMINSEE,Capital_Lit_MCO,Capital_Bloc,Capital_imagerie,Capital_Examens_biologiques,Capital_telemed
2,130783665,6.971615e+04,3,13,13055,49.0,5.0,6.0,71043.0,0
3,130785652,1.618811e+06,3,13,13055,678.0,29.0,14.0,341422.0,0
4,130786742,2.939701e+05,1,13,13055,132.0,8.0,8.0,596640.0,0
5,140000555,2.549969e+05,3,14,14118,149.0,6.0,9.0,326554.0,0
7,290000728,1.410810e+05,1,29,29019,76.0,4.0,5.0,317951.0,0
...,...,...,...,...,...,...,...,...,...,...
391,710781568,1.555003e+04,1,71,71047,22.0,0.0,0.0,0.0,0
405,740781190,5.098133e+03,1,74,74284,20.0,0.0,0.0,0.0,0
417,760780254,3.318720e+03,1,76,76758,15.0,0.0,0.0,0.0,0
436,790019491,2.939968e+04,1,79,79270,16.0,0.0,0.0,0.0,0


In [79]:
Var_CTRL = pd.concat([Var_CTRL_TELEMED_1,fi_ss_telemed_1])
Var_CTRL= Var_CTRL.drop(['FI','FI_EJ'], axis=1)

In [80]:
Var_CTRL

,Numero_finess,Points_ISA,stjr,dep,COMINSEE,Capital_Lit_MCO,Capital_Bloc,Capital_imagerie,Capital_Examens_biologiques,Capital_telemed
0,060000528,356892.030000,3,06,06088,128.0,8.0,9.0,909.0,1.0
1,130001647,854787.985000,3,13,13055,271.0,12.0,13.0,16870.0,0.0
2,210987731,205661.893333,3,21,21231,131.0,8.0,11.0,280257.0,0.0
3,310782347,170147.100000,3,31,31555,94.0,10.0,8.0,435987.0,1.0
4,350002812,61024.268333,3,35,35238,54.0,4.0,9.0,140425.0,2.0
...,...,...,...,...,...,...,...,...,...,...
391,710781568,15550.033333,1,71,71047,22.0,0.0,0.0,0.0,0.0
405,740781190,5098.133333,1,74,74284,20.0,0.0,0.0,0.0,0.0
417,760780254,3318.720000,1,76,76758,15.0,0.0,0.0,0.0,0.0
436,790019491,29399.683333,1,79,79270,16.0,0.0,0.0,0.0,0.0


## Les séjours 

### Nombre de séjours

In [81]:
MCO[['FI','FI_EJ','SEJHC_MED', 'SEJHC_CHI', 'SEJHC_OBS','SEJHC_MCO']]

,FI,FI_EJ,SEJHC_MED,SEJHC_CHI,SEJHC_OBS,SEJHC_MCO
0,010000024,010780054,13820,3937,3903.0,21660
1,010000032,010780062,3964,956,606.0,5526
2,010000065,010780096,2341,0,0.0,2341
3,010000099,010780120,135,0,0.0,135
4,010000107,010780138,164,0,0.0,164
...,...,...,...,...,...,...
1635,970462081,970400255,129,4013,0.0,4142
1636,970462107,970400305,8883,7205,2714.0,18802
1637,970466751,970400446,0,0,0.0,0
1638,980500011,980500003,7639,3712,12180.0,23531


In [82]:
sejours_fi = pd.merge( Var_CTRL, MCO[['FI','SEJHC_MED', 'SEJHC_CHI', 'SEJHC_OBS','SEJHC_MCO']], left_on = 'Numero_finess', right_on = 'FI', how= 'inner' )
sejours_fi_ej = pd.merge( Var_CTRL, MCO[['FI_EJ','SEJHC_MED', 'SEJHC_CHI', 'SEJHC_OBS','SEJHC_MCO']], left_on = 'Numero_finess', right_on = 'FI_EJ', how= 'inner' )

In [83]:
sejours_fi_ej

,Numero_finess,Points_ISA,stjr,dep,COMINSEE,Capital_Lit_MCO,Capital_Bloc,Capital_imagerie,Capital_Examens_biologiques,Capital_telemed,FI_EJ,SEJHC_MED,SEJHC_CHI,SEJHC_OBS,SEJHC_MCO
0,010780054,787929.068333,1,01,01451,369.0,12.0,10.0,954696.0,2.0,010780054,13820,3937,3903.0,21660
1,020000063,867100.583333,1,02,02691,395.0,19.0,12.0,1492636.0,4.0,020000063,18527,4998,1862.0,25387
2,020000261,533474.643333,1,02,02722,271.0,8.0,9.0,771938.0,0.0,020000261,11452,1263,2247.0,14962
3,020004404,373304.290000,1,02,02168,168.0,6.0,6.0,594902.0,1.0,020004404,7894,1106,864.0,9864
4,030780092,554335.088333,1,03,03190,311.0,7.0,9.0,714739.0,4.0,030780092,9901,3409,1429.0,14739
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
743,740781190,5098.133333,1,74,74284,20.0,0.0,0.0,0.0,0.0,740781190,322,0,0.0,322
744,760780254,3318.720000,1,76,76758,15.0,0.0,0.0,0.0,0.0,760780254,341,0,0.0,341
745,790019491,29399.683333,1,79,79270,16.0,0.0,0.0,0.0,0.0,790019491,580,0,0.0,580
746,790019491,29399.683333,1,79,79270,16.0,0.0,0.0,0.0,0.0,790019491,319,0,0.0,319


In [84]:
sejours_fi_ej_unique= sejours_fi_ej.groupby('Numero_finess').mean()
sejours_fi_ej = sejours_fi_ej.drop(['SEJHC_MED','SEJHC_CHI', 'SEJHC_OBS','SEJHC_MCO'], axis=1)
sejours_fi_ej.drop_duplicates(subset= 'Numero_finess', keep = 'first', inplace=True)
sejours_fi_ej_final = pd.merge(sejours_fi_ej, sejours_fi_ej_unique[['SEJHC_MED','SEJHC_CHI','SEJHC_OBS','SEJHC_MCO']], left_on='Numero_finess', right_on=sejours_fi_ej_unique.index  ) 

In [85]:
sejours_fi_ej_final

,Numero_finess,Points_ISA,stjr,dep,COMINSEE,Capital_Lit_MCO,Capital_Bloc,Capital_imagerie,Capital_Examens_biologiques,Capital_telemed,FI_EJ,SEJHC_MED,SEJHC_CHI,SEJHC_OBS,SEJHC_MCO
0,010780054,787929.068333,1,01,01451,369.0,12.0,10.0,954696.0,2.0,010780054,13820.0,3937.0,3903.0,21660.0
1,020000063,867100.583333,1,02,02691,395.0,19.0,12.0,1492636.0,4.0,020000063,18527.0,4998.0,1862.0,25387.0
2,020000261,533474.643333,1,02,02722,271.0,8.0,9.0,771938.0,0.0,020000261,11452.0,1263.0,2247.0,14962.0
3,020004404,373304.290000,1,02,02168,168.0,6.0,6.0,594902.0,1.0,020004404,7894.0,1106.0,864.0,9864.0
4,030780092,554335.088333,1,03,03190,311.0,7.0,9.0,714739.0,4.0,030780092,9901.0,3409.0,1429.0,14739.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
520,710781568,15550.033333,1,71,71047,22.0,0.0,0.0,0.0,0.0,710781568,523.0,0.0,0.0,523.0
521,740781190,5098.133333,1,74,74284,20.0,0.0,0.0,0.0,0.0,740781190,322.0,0.0,0.0,322.0
522,760780254,3318.720000,1,76,76758,15.0,0.0,0.0,0.0,0.0,760780254,341.0,0.0,0.0,341.0
523,790019491,29399.683333,1,79,79270,16.0,0.0,0.0,0.0,0.0,790019491,449.5,0.0,0.0,449.5


In [86]:
Var_CTRL= pd.concat([sejours_fi, sejours_fi_ej_final])

In [87]:
Var_CTRL= Var_CTRL.drop(['FI','FI_EJ'],axis=1)

In [88]:
Var_CTRL

,Numero_finess,Points_ISA,stjr,dep,COMINSEE,Capital_Lit_MCO,Capital_Bloc,Capital_imagerie,Capital_Examens_biologiques,Capital_telemed,SEJHC_MED,SEJHC_CHI,SEJHC_OBS,SEJHC_MCO
0,060000528,356892.030000,3,06,06088,128.0,8.0,9.0,909.0,1.0,4740.0,2280.0,0.0,7020.0
1,130001647,854787.985000,3,13,13055,271.0,12.0,13.0,16870.0,0.0,11409.0,3819.0,0.0,15228.0
2,210987731,205661.893333,3,21,21231,131.0,8.0,11.0,280257.0,0.0,8123.0,1704.0,0.0,9827.0
3,310782347,170147.100000,3,31,31555,94.0,10.0,8.0,435987.0,1.0,3375.0,1876.0,0.0,5251.0
4,350002812,61024.268333,3,35,35238,54.0,4.0,9.0,140425.0,2.0,1983.0,626.0,0.0,2609.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
520,710781568,15550.033333,1,71,71047,22.0,0.0,0.0,0.0,0.0,523.0,0.0,0.0,523.0
521,740781190,5098.133333,1,74,74284,20.0,0.0,0.0,0.0,0.0,322.0,0.0,0.0,322.0
522,760780254,3318.720000,1,76,76758,15.0,0.0,0.0,0.0,0.0,341.0,0.0,0.0,341.0
523,790019491,29399.683333,1,79,79270,16.0,0.0,0.0,0.0,0.0,449.5,0.0,0.0,449.5


## Données environnementales :

## Taux de chomage par département

In [89]:
# Prétraitement de la base tx_chomage
tx_chomage = tx_chomage.rename(columns={"Taux de chômage localisés par sexe et âge en moyenne annuelle en 2019 : comparaisons départementales":"Code_Departement","Unnamed: 1":"Departement","Unnamed: 2":"Taux_chomage_par_dep"})
tx_chomage = tx_chomage[['Code_Departement','Departement', 'Taux_chomage_par_dep']]
tx_chomage.drop([0,1,2,99,100, 105,106, 107],0,inplace=True)
# On ajoute Mayotte qui n'est pas dans la base tx_chomage
tx_chomage.loc[105]=[ '976', 'Mayotte', '30' ]

In [90]:
tx_chomage

,Code_Departement,Departement,Taux_chomage_par_dep
3,01,Ain,6.1
4,02,Aisne,11.8
5,03,Allier,9.1
6,04,Alpes-de-Haute-Provence,10.3
7,05,Hautes-Alpes,7.9
...,...,...,...
101,971,Guadeloupe,20.5
102,972,Martinique,14.9
103,973,Guyane,19.9
104,974,La Réunion,21.3


In [91]:
# On recode pour avoir des codes similaires ( notamment pour les DOM )
tx_chomage['Code_Departement']= tx_chomage['Code_Departement'].replace(['971', '972', '973','974','976'],['9A', '9B', '9C', '9D','9F'])

In [92]:
Var_CTRL

,Numero_finess,Points_ISA,stjr,dep,COMINSEE,Capital_Lit_MCO,Capital_Bloc,Capital_imagerie,Capital_Examens_biologiques,Capital_telemed,SEJHC_MED,SEJHC_CHI,SEJHC_OBS,SEJHC_MCO
0,060000528,356892.030000,3,06,06088,128.0,8.0,9.0,909.0,1.0,4740.0,2280.0,0.0,7020.0
1,130001647,854787.985000,3,13,13055,271.0,12.0,13.0,16870.0,0.0,11409.0,3819.0,0.0,15228.0
2,210987731,205661.893333,3,21,21231,131.0,8.0,11.0,280257.0,0.0,8123.0,1704.0,0.0,9827.0
3,310782347,170147.100000,3,31,31555,94.0,10.0,8.0,435987.0,1.0,3375.0,1876.0,0.0,5251.0
4,350002812,61024.268333,3,35,35238,54.0,4.0,9.0,140425.0,2.0,1983.0,626.0,0.0,2609.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
520,710781568,15550.033333,1,71,71047,22.0,0.0,0.0,0.0,0.0,523.0,0.0,0.0,523.0
521,740781190,5098.133333,1,74,74284,20.0,0.0,0.0,0.0,0.0,322.0,0.0,0.0,322.0
522,760780254,3318.720000,1,76,76758,15.0,0.0,0.0,0.0,0.0,341.0,0.0,0.0,341.0
523,790019491,29399.683333,1,79,79270,16.0,0.0,0.0,0.0,0.0,449.5,0.0,0.0,449.5


In [93]:
Var_CTRL = pd.merge (Var_CTRL, tx_chomage, left_on='dep', right_on='Code_Departement',how='inner')
Var_CTRL= Var_CTRL.drop(['Code_Departement'], axis=1) 

In [94]:
Var_CTRL

,Numero_finess,Points_ISA,stjr,dep,COMINSEE,Capital_Lit_MCO,Capital_Bloc,Capital_imagerie,Capital_Examens_biologiques,Capital_telemed,SEJHC_MED,SEJHC_CHI,SEJHC_OBS,SEJHC_MCO,Departement,Taux_chomage_par_dep
0,060000528,356892.030000,3,06,06088,128.0,8.0,9.0,909.0,1.0,4740.0,2280.0,0.0,7020.0,Alpes-Maritimes,8.9
1,060780491,372410.435000,3,06,06123,128.0,14.0,10.0,0.0,1.0,3480.0,4090.0,0.0,7570.0,Alpes-Maritimes,8.9
2,060780947,174552.540000,3,06,06088,105.0,5.0,4.0,0.0,2.0,7258.0,2496.0,0.0,9754.0,Alpes-Maritimes,8.9
3,060794013,238173.260000,3,06,06123,68.0,5.0,7.0,0.0,0.0,2032.0,3049.0,0.0,5081.0,Alpes-Maritimes,8.9
4,060791811,146432.633333,3,06,06088,91.0,2.0,2.0,0.0,0.0,2558.0,0.0,0.0,2558.0,Alpes-Maritimes,8.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1188,480780097,240863.268333,1,48,48095,67.0,4.5,6.0,423436.0,1.0,2052.5,1155.0,392.0,3599.5,Lozère,5.3
1189,480780162,5699.150000,1,48,48080,23.0,0.0,1.0,0.0,7.0,421.0,0.0,0.0,421.0,Lozère,5.3
1190,480780121,2396.450000,1,48,48140,22.0,0.0,0.0,0.0,0.0,205.0,0.0,0.0,205.0,Lozère,5.3
1191,480780154,517.680000,1,48,48092,20.0,0.0,0.0,0.0,0.0,384.0,0.0,0.0,384.0,Lozère,5.3


### 


Population 
Revenu fiscal moyen dans le département
Salaire horaire moyen dans le département
Dep Moyenne Salaie Cadre Horaires 

### Indicateurs de richesse 

In [95]:
indicateurs_richesse = indicateurs_richesse[['CODGEO', 'Population' ,'Dep Moyenne Salaires Horaires', 'Dep Moyenne Salaires Cadre Horaires', 'Dep Moyenne Salaires Prof Intermédiaire Horaires',
       'Dep Moyenne Salaires Employé Horaires', 'Moyenne Revenus Fiscaux Départementaux']]

In [96]:
Var_CTRL_1 = pd.merge (Var_CTRL, indicateurs_richesse, left_on='COMINSEE', right_on='CODGEO',how='inner')

In [97]:
Var_CTRL_1

,Numero_finess,Points_ISA,stjr,dep,COMINSEE,Capital_Lit_MCO,Capital_Bloc,Capital_imagerie,Capital_Examens_biologiques,Capital_telemed,...,SEJHC_MCO,Departement,Taux_chomage_par_dep,CODGEO,Population,Dep Moyenne Salaires Horaires,Dep Moyenne Salaires Cadre Horaires,Dep Moyenne Salaires Prof Intermédiaire Horaires,Dep Moyenne Salaires Employé Horaires,Moyenne Revenus Fiscaux Départementaux
0,060000528,356892.030000,3,06,06088,128.0,8.0,9.0,909.0,1.0,...,7020.0,Alpes-Maritimes,8.9,06088,343123,12.579245,22.518868,12.864151,8.835849,12288
1,060780947,174552.540000,3,06,06088,105.0,5.0,4.0,0.0,2.0,...,9754.0,Alpes-Maritimes,8.9,06088,343123,12.579245,22.518868,12.864151,8.835849,12288
2,060791811,146432.633333,3,06,06088,91.0,2.0,2.0,0.0,0.0,...,2558.0,Alpes-Maritimes,8.9,06088,343123,12.579245,22.518868,12.864151,8.835849,12288
3,060780442,22842.380000,2,06,06088,30.0,6.0,1.0,0.0,0.0,...,1605.0,Alpes-Maritimes,8.9,06088,343123,12.579245,22.518868,12.864151,8.835849,12288
4,060780715,342051.171667,2,06,06088,256.0,26.0,10.0,0.0,0.0,...,15866.0,Alpes-Maritimes,8.9,06088,343123,12.579245,22.518868,12.864151,8.835849,12288
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1157,120780085,45665.700000,1,12,12089,34.0,5.0,3.0,235068.0,0.0,...,2056.0,Aveyron,6.4,12089,6787,10.115789,20.252632,11.963158,8.478947,9936
1158,480780097,240863.268333,1,48,48095,67.0,4.5,6.0,423436.0,1.0,...,3599.5,Lozère,5.3,48095,11792,10.075000,20.475000,11.775000,8.600000,8555
1159,480780162,5699.150000,1,48,48080,23.0,0.0,1.0,0.0,7.0,...,421.0,Lozère,5.3,48080,3097,10.075000,20.475000,11.775000,8.600000,8555
1160,480780121,2396.450000,1,48,48140,22.0,0.0,0.0,0.0,0.0,...,205.0,Lozère,5.3,48140,4319,10.075000,20.475000,11.775000,8.600000,8555


In [98]:
liste2 = Var_CTRL.COMINSEE.unique()
liste1 = indicateurs_richesse.CODGEO.unique()
res_rich= [value for value in liste2 if value not in liste1]
res_rich

['9A112',
 '9A101',
 '9A105',
 '9A103',
 '9A113',
 '9A117',
 '9A127',
 '9A123',
 '9B209',
 '9B229',
 '9B217',
 '9C302',
 '9C304',
 '9C311',
 '9D411',
 '9D407',
 '9D422',
 '9D415',
 '9D410',
 '9F611']

In [99]:
# On les retirera après
DOM = Var_CTRL.loc[Var_CTRL.COMINSEE.isin(res_rich)]
Var_CTRL = pd.concat([Var_CTRL_1,DOM])

In [100]:
Var_CTRL.drop('CODGEO', axis = 1 )

,Numero_finess,Points_ISA,stjr,dep,COMINSEE,Capital_Lit_MCO,Capital_Bloc,Capital_imagerie,Capital_Examens_biologiques,Capital_telemed,...,SEJHC_OBS,SEJHC_MCO,Departement,Taux_chomage_par_dep,Population,Dep Moyenne Salaires Horaires,Dep Moyenne Salaires Cadre Horaires,Dep Moyenne Salaires Prof Intermédiaire Horaires,Dep Moyenne Salaires Employé Horaires,Moyenne Revenus Fiscaux Départementaux
0,060000528,3.568920e+05,3,06,06088,128.0,8.0,9.0,909.0,1.000000,...,0.0,7020.0,Alpes-Maritimes,8.9,343123.0,12.579245,22.518868,12.864151,8.835849,12288.0
1,060780947,1.745525e+05,3,06,06088,105.0,5.0,4.0,0.0,2.000000,...,0.0,9754.0,Alpes-Maritimes,8.9,343123.0,12.579245,22.518868,12.864151,8.835849,12288.0
2,060791811,1.464326e+05,3,06,06088,91.0,2.0,2.0,0.0,0.000000,...,0.0,2558.0,Alpes-Maritimes,8.9,343123.0,12.579245,22.518868,12.864151,8.835849,12288.0
3,060780442,2.284238e+04,2,06,06088,30.0,6.0,1.0,0.0,0.000000,...,0.0,1605.0,Alpes-Maritimes,8.9,343123.0,12.579245,22.518868,12.864151,8.835849,12288.0
4,060780715,3.420512e+05,2,06,06088,256.0,26.0,10.0,0.0,0.000000,...,1467.0,15866.0,Alpes-Maritimes,8.9,343123.0,12.579245,22.518868,12.864151,8.835849,12288.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1108,970405411,5.646000e+02,2,9D,9D411,NaN,0.0,0.0,0.0,0.000000,...,0.0,0.0,La Réunion,21.3,NaN,NaN,NaN,NaN,NaN,NaN
1109,970408589,2.697187e+06,1,9D,9D411,245.6,22.5,13.8,1600239.5,1.333333,...,2155.0,13505.8,La Réunion,21.3,NaN,NaN,NaN,NaN,NaN,NaN
1110,970421038,5.785114e+05,1,9D,9D415,276.0,7.0,8.0,895633.0,0.000000,...,3107.0,19400.0,La Réunion,21.3,NaN,NaN,NaN,NaN,NaN,NaN
1111,970403606,3.325696e+05,1,9D,9D410,177.0,6.0,0.0,0.0,0.000000,...,2572.0,12667.0,La Réunion,21.3,NaN,NaN,NaN,NaN,NaN,NaN


### Parti politique par commune 

In [101]:
# Prétraitement : 
presidentielles= presidentielles[['Code du département', 'Libellé du département', 'Code de la commune','Libellé de la commune', 'N°Panneau', 'Nom', 'Prénom']]
presidentielles

,Code du département,Libellé du département,Code de la commune,Libellé de la commune,N°Panneau,Nom,Prénom
0,1,Ain,1,L'Abergement-Clémenciat,2,LE PEN,Marine
1,1,Ain,2,L'Abergement-de-Varey,2,LE PEN,Marine
2,1,Ain,4,Ambérieu-en-Bugey,2,LE PEN,Marine
3,1,Ain,5,Ambérieux-en-Dombes,2,LE PEN,Marine
4,1,Ain,6,Ambléon,9,MÉLENCHON,Jean-Luc
...,...,...,...,...,...,...,...
35714,ZZ,Français établis hors de France,226,Wuhan,3,MACRON,Emmanuel
35715,ZZ,Français établis hors de France,227,Yaounde,3,MACRON,Emmanuel
35716,ZZ,Français établis hors de France,228,Zagreb,3,MACRON,Emmanuel
35717,ZZ,Français établis hors de France,229,Zurich,3,MACRON,Emmanuel


In [102]:
presidentielles= presidentielles[['Code du département', 'Libellé du département', 'Code de la commune','Libellé de la commune', 'N°Panneau', 'Nom', 'Prénom']]
presidentielles = presidentielles.drop(presidentielles.loc[presidentielles['Code du département'].isin(['ZS','ZW','ZP','ZN','ZP', 'ZX','ZZ'])].index)

In [103]:
presidentielles

,Code du département,Libellé du département,Code de la commune,Libellé de la commune,N°Panneau,Nom,Prénom
0,1,Ain,1,L'Abergement-Clémenciat,2,LE PEN,Marine
1,1,Ain,2,L'Abergement-de-Varey,2,LE PEN,Marine
2,1,Ain,4,Ambérieu-en-Bugey,2,LE PEN,Marine
3,1,Ain,5,Ambérieux-en-Dombes,2,LE PEN,Marine
4,1,Ain,6,Ambléon,9,MÉLENCHON,Jean-Luc
...,...,...,...,...,...,...,...
35405,ZM,Mayotte,513,M'Tsangamouji,11,FILLON,François
35406,ZM,Mayotte,514,Ouangani,11,FILLON,François
35407,ZM,Mayotte,515,Pamandzi,2,LE PEN,Marine
35408,ZM,Mayotte,516,Sada,11,FILLON,François


In [104]:
presidentielles['Code du département']= presidentielles['Code du département'].replace(['2A','2B', 'ZA', 'ZB', 'ZC', 'ZD', 'ZM'],['2A','2B', '9A', '9B', '9C', '9D', '9F'])

In [105]:
nom_parti = pd.DataFrame({"Nom": presidentielles['Nom'].unique(), 'Bloc_politique': ['Droite à extrême droite','Extrême gauche à gauche','Centre gauche à centre droit','Centre droit à droite' , 'Droite à extrême droite', 'Centre gauche à centre droit', 'Gauche à centre gauche', 'Droite à extrême droite', 'Extrême gauche à gauche'] })
nom_parti

,Nom,Bloc_politique
0,LE PEN,Droite à extrême droite
1,MÉLENCHON,Extrême gauche à gauche
2,MACRON,Centre gauche à centre droit
3,FILLON,Centre droit à droite
4,DUPONT-AIGNAN,Droite à extrême droite
5,LASSALLE,Centre gauche à centre droit
6,HAMON,Gauche à centre gauche
7,ASSELINEAU,Droite à extrême droite
8,POUTOU,Extrême gauche à gauche


In [106]:
parti_commune = pd.merge (presidentielles, nom_parti , left_on='Nom', right_on='Nom',how='inner')
parti_commune

,Code du département,Libellé du département,Code de la commune,Libellé de la commune,N°Panneau,Nom,Prénom,Bloc_politique
0,1,Ain,1,L'Abergement-Clémenciat,2,LE PEN,Marine,Droite à extrême droite
1,1,Ain,2,L'Abergement-de-Varey,2,LE PEN,Marine,Droite à extrême droite
2,1,Ain,4,Ambérieu-en-Bugey,2,LE PEN,Marine,Droite à extrême droite
3,1,Ain,5,Ambérieux-en-Dombes,2,LE PEN,Marine,Droite à extrême droite
4,1,Ain,7,Ambronay,2,LE PEN,Marine,Droite à extrême droite
...,...,...,...,...,...,...,...,...
35405,64,Pyrénées-Atlantiques,383,Mialos,4,HAMON,Benoît,Gauche à centre gauche
35406,65,Hautes-Pyrénées,277,Lombrès,4,HAMON,Benoît,Gauche à centre gauche
35407,9D,La Réunion,412,Saint-Joseph,4,HAMON,Benoît,Gauche à centre gauche
35408,11,Aude,377,Serres,10,ASSELINEAU,François,Droite à extrême droite


In [107]:
parti_commune['Code de la commune'] =parti_commune['Code de la commune'].apply(str)
parti_commune['Code de la commune'] =parti_commune['Code de la commune'].str.zfill(3)
parti_commune['Code du département'] =parti_commune['Code du département'].apply(str)
parti_commune['Code du département'] =parti_commune['Code du département'].str.zfill(2)

In [108]:
parti_commune

,Code du département,Libellé du département,Code de la commune,Libellé de la commune,N°Panneau,Nom,Prénom,Bloc_politique
0,01,Ain,001,L'Abergement-Clémenciat,2,LE PEN,Marine,Droite à extrême droite
1,01,Ain,002,L'Abergement-de-Varey,2,LE PEN,Marine,Droite à extrême droite
2,01,Ain,004,Ambérieu-en-Bugey,2,LE PEN,Marine,Droite à extrême droite
3,01,Ain,005,Ambérieux-en-Dombes,2,LE PEN,Marine,Droite à extrême droite
4,01,Ain,007,Ambronay,2,LE PEN,Marine,Droite à extrême droite
...,...,...,...,...,...,...,...,...
35405,64,Pyrénées-Atlantiques,383,Mialos,4,HAMON,Benoît,Gauche à centre gauche
35406,65,Hautes-Pyrénées,277,Lombrès,4,HAMON,Benoît,Gauche à centre gauche
35407,9D,La Réunion,412,Saint-Joseph,4,HAMON,Benoît,Gauche à centre gauche
35408,11,Aude,377,Serres,10,ASSELINEAU,François,Droite à extrême droite


In [109]:
parti_commune['Code_INSEE'] = parti_commune['Code du département']+parti_commune['Code de la commune']
parti_commune

,Code du département,Libellé du département,Code de la commune,Libellé de la commune,N°Panneau,Nom,Prénom,Bloc_politique,Code_INSEE
0,01,Ain,001,L'Abergement-Clémenciat,2,LE PEN,Marine,Droite à extrême droite,01001
1,01,Ain,002,L'Abergement-de-Varey,2,LE PEN,Marine,Droite à extrême droite,01002
2,01,Ain,004,Ambérieu-en-Bugey,2,LE PEN,Marine,Droite à extrême droite,01004
3,01,Ain,005,Ambérieux-en-Dombes,2,LE PEN,Marine,Droite à extrême droite,01005
4,01,Ain,007,Ambronay,2,LE PEN,Marine,Droite à extrême droite,01007
...,...,...,...,...,...,...,...,...,...
35405,64,Pyrénées-Atlantiques,383,Mialos,4,HAMON,Benoît,Gauche à centre gauche,64383
35406,65,Hautes-Pyrénées,277,Lombrès,4,HAMON,Benoît,Gauche à centre gauche,65277
35407,9D,La Réunion,412,Saint-Joseph,4,HAMON,Benoît,Gauche à centre gauche,9D412
35408,11,Aude,377,Serres,10,ASSELINEAU,François,Droite à extrême droite,11377


In [110]:
Test = Var_CTRL.copy()

In [111]:
Test = pd.merge (Var_CTRL, parti_commune[['Bloc_politique','Code_INSEE']], left_on='COMINSEE', right_on='Code_INSEE', how='inner')
Test

,Numero_finess,Points_ISA,stjr,dep,COMINSEE,Capital_Lit_MCO,Capital_Bloc,Capital_imagerie,Capital_Examens_biologiques,Capital_telemed,...,Taux_chomage_par_dep,CODGEO,Population,Dep Moyenne Salaires Horaires,Dep Moyenne Salaires Cadre Horaires,Dep Moyenne Salaires Prof Intermédiaire Horaires,Dep Moyenne Salaires Employé Horaires,Moyenne Revenus Fiscaux Départementaux,Bloc_politique,Code_INSEE
0,060000528,356892.030000,3,06,06088,128.0,8.0,9.0,909.0,1.0,...,8.9,06088,343123.0,12.579245,22.518868,12.864151,8.835849,12288.0,Centre droit à droite,06088
1,060780947,174552.540000,3,06,06088,105.0,5.0,4.0,0.0,2.0,...,8.9,06088,343123.0,12.579245,22.518868,12.864151,8.835849,12288.0,Centre droit à droite,06088
2,060791811,146432.633333,3,06,06088,91.0,2.0,2.0,0.0,0.0,...,8.9,06088,343123.0,12.579245,22.518868,12.864151,8.835849,12288.0,Centre droit à droite,06088
3,060780442,22842.380000,2,06,06088,30.0,6.0,1.0,0.0,0.0,...,8.9,06088,343123.0,12.579245,22.518868,12.864151,8.835849,12288.0,Centre droit à droite,06088
4,060780715,342051.171667,2,06,06088,256.0,26.0,10.0,0.0,0.0,...,8.9,06088,343123.0,12.579245,22.518868,12.864151,8.835849,12288.0,Centre droit à droite,06088
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1185,970466751,7336.790000,2,9D,9D407,NaN,NaN,0.0,0.0,0.0,...,21.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Extrême gauche à gauche,9D407
1186,970462073,81023.520000,2,9D,9D422,69.0,9.0,0.0,0.0,0.0,...,21.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Droite à extrême droite,9D422
1187,970404109,4451.235000,2,9D,9D415,20.0,0.0,0.0,0.0,0.0,...,21.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Extrême gauche à gauche,9D415
1188,970421038,578511.420000,1,9D,9D415,276.0,7.0,8.0,895633.0,0.0,...,21.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Extrême gauche à gauche,9D415


Il manque 3 lignes: Les établissement ci-dessous 

In [112]:
list2 = Var_CTRL.Numero_finess.unique()
list1 = Test.Numero_finess.unique()
res = [value for value in list2 if value not in list1]
res

['970100186', '970100160', '980500003']

In [113]:
Var_CTRL[Var_CTRL.Numero_finess=='970100160']

,Numero_finess,Points_ISA,stjr,dep,COMINSEE,Capital_Lit_MCO,Capital_Bloc,Capital_imagerie,Capital_Examens_biologiques,Capital_telemed,...,SEJHC_MCO,Departement,Taux_chomage_par_dep,CODGEO,Population,Dep Moyenne Salaires Horaires,Dep Moyenne Salaires Cadre Horaires,Dep Moyenne Salaires Prof Intermédiaire Horaires,Dep Moyenne Salaires Employé Horaires,Moyenne Revenus Fiscaux Départementaux
812,970100160,3433.26,1,9A,9A123,10.0,0.0,0.0,0.0,2.0,...,980.0,Guadeloupe,20.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [114]:
Var_CTRL[Var_CTRL.Numero_finess=='970100186']

,Numero_finess,Points_ISA,stjr,dep,COMINSEE,Capital_Lit_MCO,Capital_Bloc,Capital_imagerie,Capital_Examens_biologiques,Capital_telemed,...,SEJHC_MCO,Departement,Taux_chomage_par_dep,CODGEO,Population,Dep Moyenne Salaires Horaires,Dep Moyenne Salaires Cadre Horaires,Dep Moyenne Salaires Prof Intermédiaire Horaires,Dep Moyenne Salaires Employé Horaires,Moyenne Revenus Fiscaux Départementaux
811,970100186,54908.665,1,9A,9A127,71.0,3.0,2.0,0.0,2.5,...,4306.0,Guadeloupe,20.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [115]:
parti_commune[parti_commune['Code du département']=='9A']

,Code du département,Libellé du département,Code de la commune,Libellé de la commune,N°Panneau,Nom,Prénom,Bloc_politique,Code_INSEE
22445,9A,Guadeloupe,106,Bouillante,9,MÉLENCHON,Jean-Luc,Extrême gauche à gauche,9A106
22446,9A,Guadeloupe,110,La Désirade,9,MÉLENCHON,Jean-Luc,Extrême gauche à gauche,9A110
22447,9A,Guadeloupe,115,Lamentin,9,MÉLENCHON,Jean-Luc,Extrême gauche à gauche,9A115
22448,9A,Guadeloupe,134,Vieux-Habitants,9,MÉLENCHON,Jean-Luc,Extrême gauche à gauche,9A134
29697,9A,Guadeloupe,101,Les Abymes,3,MACRON,Emmanuel,Centre gauche à centre droit,9A101
29698,9A,Guadeloupe,102,Anse-Bertrand,3,MACRON,Emmanuel,Centre gauche à centre droit,9A102
29699,9A,Guadeloupe,103,Baie-Mahault,3,MACRON,Emmanuel,Centre gauche à centre droit,9A103
29700,9A,Guadeloupe,104,Baillif,3,MACRON,Emmanuel,Centre gauche à centre droit,9A104
29701,9A,Guadeloupe,105,Basse-Terre,3,MACRON,Emmanuel,Centre gauche à centre droit,9A105
29702,9A,Guadeloupe,107,Capesterre-Belle-Eau,3,MACRON,Emmanuel,Centre gauche à centre droit,9A107


Effectivement, les codes INSEE des communes 9A127 et 9A123 sont absents. Cependant, en Guadeloupe, les votes sont majoritairement en faveur du parti Centre gauche à Centre droit. Nous supposons que c'est le cas pour ces deux communes également. 

In [116]:
Test= Test.append(Var_CTRL[Var_CTRL.Numero_finess=='970100160'])
Test= Test.append(Var_CTRL[Var_CTRL.Numero_finess=='970100186'])

In [117]:
Test['Bloc_politique'] = Test['Bloc_politique'].fillna(value='Centre gauche à centre droit')

In [118]:
parti_commune[parti_commune['Code du département']=='9F']

,Code du département,Libellé du département,Code de la commune,Libellé de la commune,N°Panneau,Nom,Prénom,Bloc_politique,Code_INSEE
18995,9F,Mayotte,501,Acoua,2,LE PEN,Marine,Droite à extrême droite,9F501
18996,9F,Mayotte,502,Bandraboua,2,LE PEN,Marine,Droite à extrême droite,9F502
18997,9F,Mayotte,504,Bouéni,2,LE PEN,Marine,Droite à extrême droite,9F504
18998,9F,Mayotte,515,Pamandzi,2,LE PEN,Marine,Droite à extrême droite,9F515
29740,9F,Mayotte,508,Dzaoudzi,3,MACRON,Emmanuel,Centre gauche à centre droit,9F508
29741,9F,Mayotte,511,Mamoudzou,3,MACRON,Emmanuel,Centre gauche à centre droit,9F511
35279,9F,Mayotte,503,Bandrele,11,FILLON,François,Centre droit à droite,9F503
35280,9F,Mayotte,505,Chiconi,11,FILLON,François,Centre droit à droite,9F505
35281,9F,Mayotte,506,Chirongui,11,FILLON,François,Centre droit à droite,9F506
35282,9F,Mayotte,507,Dembeni,11,FILLON,François,Centre droit à droite,9F507


Les établissements sont plutot de "centre droit à droite" à Mayotte

In [119]:
Test= Test.append(Var_CTRL[Var_CTRL.Numero_finess=='980500003'])

In [120]:
Test['Bloc_politique'] = Test['Bloc_politique'].fillna(value='Centre droit à droite')

In [121]:
Var_CTRL = Test

In [122]:
Var_CTRL.shape

(1193, 25)

In [123]:
Var_CTRL = Var_CTRL[~Var_CTRL['Numero_finess'].astype(str).str.startswith('97')] 
Var_CTRL = Var_CTRL[~Var_CTRL['Numero_finess'].astype(str).str.startswith('98')] 

Var_CTRL

,Numero_finess,Points_ISA,stjr,dep,COMINSEE,Capital_Lit_MCO,Capital_Bloc,Capital_imagerie,Capital_Examens_biologiques,Capital_telemed,...,Taux_chomage_par_dep,CODGEO,Population,Dep Moyenne Salaires Horaires,Dep Moyenne Salaires Cadre Horaires,Dep Moyenne Salaires Prof Intermédiaire Horaires,Dep Moyenne Salaires Employé Horaires,Moyenne Revenus Fiscaux Départementaux,Bloc_politique,Code_INSEE
0,060000528,356892.030000,3,06,06088,128.0,8.0,9.0,909.0,1.0,...,8.9,06088,343123.0,12.579245,22.518868,12.864151,8.835849,12288.0,Centre droit à droite,06088
1,060780947,174552.540000,3,06,06088,105.0,5.0,4.0,0.0,2.0,...,8.9,06088,343123.0,12.579245,22.518868,12.864151,8.835849,12288.0,Centre droit à droite,06088
2,060791811,146432.633333,3,06,06088,91.0,2.0,2.0,0.0,0.0,...,8.9,06088,343123.0,12.579245,22.518868,12.864151,8.835849,12288.0,Centre droit à droite,06088
3,060780442,22842.380000,2,06,06088,30.0,6.0,1.0,0.0,0.0,...,8.9,06088,343123.0,12.579245,22.518868,12.864151,8.835849,12288.0,Centre droit à droite,06088
4,060780715,342051.171667,2,06,06088,256.0,26.0,10.0,0.0,0.0,...,8.9,06088,343123.0,12.579245,22.518868,12.864151,8.835849,12288.0,Centre droit à droite,06088
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1157,120780085,45665.700000,1,12,12089,34.0,5.0,3.0,235068.0,0.0,...,6.4,12089,6787.0,10.115789,20.252632,11.963158,8.478947,9936.0,Extrême gauche à gauche,12089
1158,480780097,240863.268333,1,48,48095,67.0,4.5,6.0,423436.0,1.0,...,5.3,48095,11792.0,10.075000,20.475000,11.775000,8.600000,8555.0,Centre gauche à centre droit,48095
1159,480780162,5699.150000,1,48,48080,23.0,0.0,1.0,0.0,7.0,...,5.3,48080,3097.0,10.075000,20.475000,11.775000,8.600000,8555.0,Centre gauche à centre droit,48080
1160,480780121,2396.450000,1,48,48140,22.0,0.0,0.0,0.0,0.0,...,5.3,48140,4319.0,10.075000,20.475000,11.775000,8.600000,8555.0,Centre droit à droite,48140


In [124]:
Var_CTRL = Var_CTRL[['Numero_finess', 'Points_ISA','stjr', 
                     
                     'dep', 'Departement','COMINSEE', 'Capital_Lit_MCO',
       'Capital_Bloc', 'Capital_imagerie', 'Capital_Examens_biologiques',
       'Capital_telemed', 'SEJHC_MED', 'SEJHC_CHI', 'SEJHC_OBS', 'SEJHC_MCO'
        
       , 'Taux_chomage_par_dep', 'Population',
       'Dep Moyenne Salaires Horaires', 'Dep Moyenne Salaires Cadre Horaires',
       'Dep Moyenne Salaires Prof Intermédiaire Horaires',
       'Dep Moyenne Salaires Employé Horaires',
       'Moyenne Revenus Fiscaux Départementaux', 'Bloc_politique']]

In [125]:
Var_CTRL.to_csv('C:/Users/louis/OneDrive/Documents/ENSAE/2A/Stat_app/Variables_controle.csv',index=False)

## Statistiques descriptives 

In [158]:
Var_CTRL.Taux_chomage_par_dep= Var_CTRL.Taux_chomage_par_dep.astype(int)

C:\Users\louis\anaconda3\lib\site-packages\pandas\core\generic.py:5168: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [160]:
Var_CTRL[['Numero_finess', 'Points_ISA', 'stjr', 'dep',
       'Capital_Lit_MCO', 'Capital_Bloc', 'Capital_imagerie',
       'Capital_Examens_biologiques','Taux_chomage_par_dep',
       'Population', 'Dep Moyenne Salaires Horaires',
       'Dep Moyenne Salaires Cadre Horaires',
       'Dep Moyenne Salaires Prof Intermédiaire Horaires',
       'Dep Moyenne Salaires Employé Horaires',
       'Moyenne Revenus Fiscaux Départementaux', 'Bloc_politique']].corr()

,Points_ISA,stjr,Capital_Lit_MCO,Capital_Bloc,Capital_imagerie,Capital_Examens_biologiques,Taux_chomage_par_dep,Population,Dep Moyenne Salaires Horaires,Dep Moyenne Salaires Cadre Horaires,Dep Moyenne Salaires Prof Intermédiaire Horaires,Dep Moyenne Salaires Employé Horaires,Moyenne Revenus Fiscaux Départementaux
Points_ISA,1.000000,-0.231872,0.657320,0.537638,0.681035,0.764178,0.012325,-0.014342,-0.019076,-0.037890,-0.000464,0.002838,-0.021237
stjr,-0.231872,1.000000,-0.196501,0.045532,-0.217664,-0.299215,0.007564,0.241235,0.251533,0.216728,0.249935,0.185680,0.138475
Capital_Lit_MCO,0.657320,-0.196501,1.000000,0.720443,0.766204,0.675850,0.042300,-0.009487,-0.017533,-0.041729,0.000799,0.011943,-0.030489
Capital_Bloc,0.537638,0.045532,0.720443,1.000000,0.600412,0.458229,0.029644,0.099673,0.077843,0.048890,0.076205,0.058307,0.049509
Capital_imagerie,0.681035,-0.217664,0.766204,0.600412,1.000000,0.665984,0.036285,-0.006132,-0.005064,-0.031265,0.002588,0.008948,0.002173
Capital_Examens_biologiques,0.764178,-0.299215,0.675850,0.458229,0.665984,1.000000,0.029372,-0.031189,-0.018891,-0.039848,-0.007597,0.014935,-0.028074
Taux_chomage_par_dep,0.012325,0.007564,0.042300,0.029644,0.036285,0.029372,1.000000,-0.160714,-0.316034,-0.305901,-0.215728,-0.367575,-0.497634
Population,-0.014342,0.241235,-0.009487,0.099673,-0.006132,-0.031189,-0.160714,1.000000,0.609209,0.657543,0.434587,0.329663,0.465794
Dep Moyenne Salaires Horaires,-0.019076,0.251533,-0.017533,0.077843,-0.005064,-0.018891,-0.316034,0.609209,1.000000,0.948660,0.916252,0.863198,0.661572
Dep Moyenne Salaires Cadre Horaires,-0.037890,0.216728,-0.041729,0.048890,-0.031265,-0.039848,-0.305901,0.657543,0.948660,1.000000,0.848362,0.759894,0.664567


In [161]:
Var_CTRL.describe ()

,Points_ISA,stjr,Capital_Lit_MCO,Capital_Bloc,Capital_imagerie,Capital_Examens_biologiques,Capital_telemed,SEJHC_MED,SEJHC_CHI,SEJHC_OBS,SEJHC_MCO,Taux_chomage_par_dep,Population,Dep Moyenne Salaires Horaires,Dep Moyenne Salaires Cadre Horaires,Dep Moyenne Salaires Prof Intermédiaire Horaires,Dep Moyenne Salaires Employé Horaires,Moyenne Revenus Fiscaux Départementaux
count,1.162000e+03,1162.000000,1139.000000,1153.000000,1162.000000,1.149000e+03,1162.000000,1162.000000,1162.000000,1161.000000,1162.000000,1162.000000,1.162000e+03,1162.000000,1162.000000,1162.000000,1162.000000,1162.000000
mean,2.939915e+05,1.707401,125.109263,7.379577,4.256971,2.508881e+05,0.728148,3953.236728,2091.528256,673.093714,6717.279443,7.869191,1.367009e+05,11.793749,21.764363,12.674821,8.783985,10648.398451
std,6.281782e+05,0.716795,127.684194,7.191520,5.047103,6.567670e+05,1.514649,5172.505167,2471.292213,1146.876402,7600.700172,1.672114,3.998901e+05,1.908702,1.674054,0.523229,0.353571,2063.805307
min,1.347900e+02,1.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,1.820000e+02,9.862500,19.750000,11.740000,8.261905,6282.000000
25%,2.993899e+04,1.000000,35.000000,1.000000,0.000000,0.000000e+00,0.000000,364.750000,0.000000,0.000000,1204.500000,7.000000,8.053750e+03,10.633333,20.775000,12.346721,8.555172,9471.000000
50%,1.016205e+05,2.000000,85.000000,6.000000,3.000000,0.000000e+00,0.000000,1963.000000,1428.000000,0.000000,4168.500000,8.000000,2.295500e+04,11.204762,21.280808,12.535714,8.700862,10239.000000
75%,2.975944e+05,2.000000,171.166667,10.500000,7.000000,6.261400e+04,1.000000,5581.250000,3168.312500,1081.000000,9749.750000,9.000000,6.171825e+04,11.981429,22.083750,12.902500,8.848113,11515.000000
max,9.116903e+06,3.000000,1055.000000,44.000000,28.000000,1.060114e+07,11.000000,39884.000000,20991.000000,7639.000000,60719.000000,13.000000,2.125851e+06,17.800000,27.600000,14.185000,9.742553,16895.000000


In [162]:
Var_CTRL[Var_CTRL.Taux_chomage_par_dep == 13 ]

,Numero_finess,Points_ISA,stjr,dep,Departement,COMINSEE,Capital_Lit_MCO,Capital_Bloc,Capital_imagerie,Capital_Examens_biologiques,...,SEJHC_OBS,SEJHC_MCO,Taux_chomage_par_dep,Population,Dep Moyenne Salaires Horaires,Dep Moyenne Salaires Cadre Horaires,Dep Moyenne Salaires Prof Intermédiaire Horaires,Dep Moyenne Salaires Employé Horaires,Moyenne Revenus Fiscaux Départementaux,Bloc_politique
781,660780784,4.057920e+05,2,66,Pyrénées-Orientales,66136,198.0,15.0,0.0,0.0,...,0.0,13308.0,13,105096.0,10.247917,20.9625,12.102083,8.347917,10400.0,Droite à extrême droite
782,660006305,2.480377e+05,3,66,Pyrénées-Orientales,66136,114.0,14.0,0.0,0.0,...,0.0,6627.0,13,105096.0,10.247917,20.9625,12.102083,8.347917,10400.0,Droite à extrême droite
783,660780669,4.830836e+04,2,66,Pyrénées-Orientales,66136,67.0,5.0,0.0,0.0,...,2843.0,3795.0,13,105096.0,10.247917,20.9625,12.102083,8.347917,10400.0,Droite à extrême droite
784,660780180,1.333064e+06,1,66,Pyrénées-Orientales,66136,636.0,17.0,17.0,2006882.0,...,3731.0,34746.0,13,105096.0,10.247917,20.9625,12.102083,8.347917,10400.0,Droite à extrême droite
785,660790387,2.687248e+05,2,66,Pyrénées-Orientales,66028,180.0,22.0,0.0,0.0,...,0.0,11029.0,13,8250.0,10.247917,20.9625,12.102083,8.347917,10400.0,Droite à extrême droite
786,660780628,4.517276e+04,2,66,Pyrénées-Orientales,66049,40.0,4.0,5.0,0.0,...,0.0,2718.0,13,7292.0,10.247917,20.9625,12.102083,8.347917,10400.0,Extrême gauche à gauche
787,660009689,5.098133e+03,3,66,Pyrénées-Orientales,66067,15.0,0.0,1.0,0.0,...,0.0,282.0,13,551.0,10.247917,20.9625,12.102083,8.347917,10400.0,Droite à extrême droite
788,660780776,2.481807e+04,2,66,Pyrénées-Orientales,66149,36.0,4.0,0.0,0.0,...,0.0,2199.0,13,5797.0,10.247917,20.9625,12.102083,8.347917,10400.0,Extrême gauche à gauche
789,660780271,5.098133e+03,1,66,Pyrénées-Orientales,66149,20.0,0.0,0.0,0.0,...,0.0,346.0,13,5797.0,10.247917,20.9625,12.102083,8.347917,10400.0,Extrême gauche à gauche


In [148]:
Var_CTRL[Var_CTRL.Capital_Lit_MCO ==1055 ][['Numero_finess', 'Points_ISA', 'stjr', 'dep', 'Departement', 'COMINSEE',
       'Capital_Lit_MCO', 'Capital_Bloc', 'Capital_imagerie',
       'Capital_Examens_biologiques', 'Capital_telemed']]

,Numero_finess,Points_ISA,stjr,dep,Departement,COMINSEE,Capital_Lit_MCO,Capital_Bloc,Capital_imagerie,Capital_Examens_biologiques,Capital_telemed
804,490000031,2.541528e+06,1,49,Maine-et-Loire,49007,1055.0,40.0,27.0,3567658.0,3.0


In [142]:
Var_CTRL[Var_CTRL.Capital_Bloc == 44 ]

,Numero_finess,Points_ISA,stjr,dep,Departement,COMINSEE,Capital_Lit_MCO,Capital_Bloc,Capital_imagerie,Capital_Examens_biologiques,...,SEJHC_OBS,SEJHC_MCO,Taux_chomage_par_dep,Population,Dep Moyenne Salaires Horaires,Dep Moyenne Salaires Cadre Horaires,Dep Moyenne Salaires Prof Intermédiaire Horaires,Dep Moyenne Salaires Employé Horaires,Moyenne Revenus Fiscaux Départementaux,Bloc_politique
191,680000973,1.737656e+06,1,68,Haut-Rhin,68066,401.0,44.0,23.0,2446043.0,...,2098.5,20572.5,8,65118.0,11.420000,21.760000,12.735000,8.688333,12258.0,Centre gauche à centre droit
604,440033819,3.077773e+05,2,44,Loire-Atlantique,44162,323.0,44.0,0.0,0.0,...,6138.0,29717.0,6.8,43728.0,11.068852,20.145902,12.346721,8.654098,11552.0,Centre gauche à centre droit


In [168]:
Var_CTRL.groupby('Bloc_politique').mean()['Points_ISA']

Bloc_politique
Centre droit à droite           198302.483721
Centre gauche à centre droit    331733.712239
Droite à extrême droite         249498.056278
Extrême gauche à gauche         327260.286551
Name: Points_ISA, dtype: float64

In [135]:
Var_CTRL.groupby('dep').count() [Var_CTRL.groupby('dep').count().Numero_finess ==49 ]

,Numero_finess,Points_ISA,stjr,Departement,COMINSEE,Capital_Lit_MCO,Capital_Bloc,Capital_imagerie,Capital_Examens_biologiques,Capital_telemed,...,SEJHC_OBS,SEJHC_MCO,Taux_chomage_par_dep,Population,Dep Moyenne Salaires Horaires,Dep Moyenne Salaires Cadre Horaires,Dep Moyenne Salaires Prof Intermédiaire Horaires,Dep Moyenne Salaires Employé Horaires,Moyenne Revenus Fiscaux Départementaux,Bloc_politique
dep,,,,,,,,,,,,,,,,,,,,,
59,49,49,49,49,49,48,49,49,48,49,...,49,49,49,49,49,49,49,49,49,49


## La qualité  ( non incluse dans les variables de contrôle)

Les données de qualité sont disponible sur le site : https://hospidiag.atih.sante.fr/cgi-bin/broker?_service=hospidiag&_debug=0&_program=hd.accueil_hd.sas
Il faut télécharger " Données brutes " puis utiliser le fichier de 2018 ( aucune valeur de qualité pour 2019 ) 

In [126]:
Hospi_diag2018 = pd.ExcelFile("C:/Users/louis/OneDrive/Documents/ENSAE/2A/Stat_app/Bases_complementaires/hospidiag_opendata_2018.xlsx")

In [127]:
print(Hospi_diag2018.sheet_names)

['Indicateurs', 'LISEZMOI', 'hd2018', 'qualite_2018', 'pdmreg_2018', 'pdmza_2018']


In [128]:
Qualite = pd.read_excel(Hospi_diag2018, 'qualite_2018')

In [129]:
Qualite

,finess,rs,reg,cat,taille_MCO,ind,indic_lib,fmt_indic,res,IC,...,typ_C,typ_D,typ_E,nat_A,nat_B,nat_C,nat_D,nat_E,finess_geo,rs_geo
0,010007300,CLINIQUE AMBULATOIRE CENDANEG,84,CLI,T1,Q1,esatismco,NaN,Non concerné,Non concerné,...,30.120482,9.638554,NaN,12.870013,28.957529,37.194337,20.978121,NaN,010007300,CLINIQUE AMBULATOIRE CENDANEG
1,010007300,CLINIQUE AMBULATOIRE CENDANEG,84,CLI,T1,Q2,esatisca,NaN,Données insuffisantes,Non concerné,...,30.136986,5.479452,NaN,15.042735,36.068376,38.632479,10.256410,NaN,010007300,CLINIQUE AMBULATOIRE CENDANEG
2,010007300,CLINIQUE AMBULATOIRE CENDANEG,84,CLI,T1,Q3,dpa_qls_mco,NaN,Non concerné,Non concerné,...,69.268293,NaN,NaN,8.460366,14.939024,76.600610,NaN,NaN,010007300,CLINIQUE AMBULATOIRE CENDANEG
3,010007300,CLINIQUE AMBULATOIRE CENDANEG,84,CLI,T1,Q4,dpa_dtn_mco,Année sans recueil,Année sans recueil,Année sans recueil,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,010007300,CLINIQUE AMBULATOIRE CENDANEG
4,010007300,CLINIQUE AMBULATOIRE CENDANEG,84,CLI,T1,Q5,dan_trd_mco,Année sans recueil,Année sans recueil,Année sans recueil,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,010007300,CLINIQUE AMBULATOIRE CENDANEG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13923,980500763,UNITÉ D'AUTODIALYSE MAMOUDZOU,6,CLI,T0,Q4,dpa_dtn_mco,Année sans recueil,Année sans recueil,Année sans recueil,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,980500763,UNITE AUTODIALYSE MAMOUDZOU
13924,980500763,UNITÉ D'AUTODIALYSE MAMOUDZOU,6,CLI,T0,Q5,dan_trd_mco,Année sans recueil,Année sans recueil,Année sans recueil,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,980500763,UNITE AUTODIALYSE MAMOUDZOU
13925,980500763,UNITÉ D'AUTODIALYSE MAMOUDZOU,6,CLI,T0,Q6,dan_tda_mco,Année sans recueil,Année sans recueil,Année sans recueil,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,980500763,UNITE AUTODIALYSE MAMOUDZOU
13926,980500763,UNITÉ D'AUTODIALYSE MAMOUDZOU,6,CLI,T0,Q7,rcp_mco,Année sans recueil,Année sans recueil,Année sans recueil,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,980500763,UNITE AUTODIALYSE MAMOUDZOU


In [130]:
Qualite.columns

Index(['finess', 'rs', 'reg', 'cat', 'taille_MCO', 'ind', 'indic_lib',
       'fmt_indic', 'res', 'IC', 'classe', 'annee', 'evo', 'cat_A', 'cat_B',
       'cat_C', 'cat_D', 'cat_E', 'reg_A', 'reg_B', 'reg_C', 'reg_D', 'reg_E',
       'typ_A', 'typ_B', 'typ_C', 'typ_D', 'typ_E', 'nat_A', 'nat_B', 'nat_C',
       'nat_D', 'nat_E', 'finess_geo', 'rs_geo'],
      dtype='object')

In [131]:
# On selectionne seulement les variables qui nous interessent
Qualite= Qualite[['finess','ind','cat_A','cat_B','cat_C','cat_D']]

On choisit de selectionner deux indicateurs : 
- La satisfaction des patients : Q1 
- L'indicateur de consommation de produits hydro-alcooliques : Q8

Q1 : Score ajusté de satisfaction globale des patients (e-Satis 48h MCO)

Q8 : Consommation des produits hydro-alcooliques version 3 (ICSHA.3)

In [132]:
# On commence par analyser le premier indicateur : Q1 
Qualite_Q1 = Qualite[Qualite['ind']=='Q1']

In [133]:
Qualite_Q1

,finess,ind,cat_A,cat_B,cat_C,cat_D
0,010007300,Q1,18.478261,41.304348,30.434783,9.782609
8,010007987,Q1,2.898551,14.975845,48.309179,33.816425
16,010007987,Q1,2.898551,14.975845,48.309179,33.816425
24,010008407,Q1,2.898551,14.975845,48.309179,33.816425
32,010009132,Q1,2.898551,14.975845,48.309179,33.816425
...,...,...,...,...,...,...
13888,970462073,Q1,18.478261,41.304348,30.434783,9.782609
13896,970462081,Q1,18.478261,41.304348,30.434783,9.782609
13904,970462107,Q1,18.478261,41.304348,30.434783,9.782609
13912,970466751,Q1,18.478261,41.304348,30.434783,9.782609


In [134]:
Qualite_Q1.finess.nunique()

1515

In [135]:
finess_double = Qualite_Q1[Qualite_Q1.duplicated(subset ='finess')]['finess']

In [136]:
Qualite_Q1[Qualite_Q1['finess'].isin(finess_double)]

,finess,ind,cat_A,cat_B,cat_C,cat_D
8,010007987,Q1,2.898551,14.975845,48.309179,33.816425
16,010007987,Q1,2.898551,14.975845,48.309179,33.816425
32,010009132,Q1,2.898551,14.975845,48.309179,33.816425
40,010009132,Q1,2.898551,14.975845,48.309179,33.816425
368,050002948,Q1,2.898551,14.975845,48.309179,33.816425
...,...,...,...,...,...,...
13824,970408589,Q1,1.666667,10.000000,47.500000,40.833333
13832,970408589,Q1,1.666667,10.000000,47.500000,40.833333
13840,970408589,Q1,1.666667,10.000000,47.500000,40.833333
13848,970408589,Q1,1.666667,10.000000,47.500000,40.833333


Les doublons ont les memes valeurs, on peut supprimer. 

In [137]:
Qualite_Q1.drop_duplicates(subset ='finess', keep = 'first', inplace=True)

<ipython-input-137-2f5225847d77>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Qualite_Q1.drop_duplicates(subset ='finess', keep = 'first', inplace=True)


In [138]:
Qualite_Q1

,finess,ind,cat_A,cat_B,cat_C,cat_D
0,010007300,Q1,18.478261,41.304348,30.434783,9.782609
8,010007987,Q1,2.898551,14.975845,48.309179,33.816425
24,010008407,Q1,2.898551,14.975845,48.309179,33.816425
32,010009132,Q1,2.898551,14.975845,48.309179,33.816425
48,010780054,Q1,2.898551,14.975845,48.309179,33.816425
...,...,...,...,...,...,...
13888,970462073,Q1,18.478261,41.304348,30.434783,9.782609
13896,970462081,Q1,18.478261,41.304348,30.434783,9.782609
13904,970462107,Q1,18.478261,41.304348,30.434783,9.782609
13912,970466751,Q1,18.478261,41.304348,30.434783,9.782609


In [139]:
df =pd.merge (Var_CTRL,Qualite_Q1, left_on='Numero_finess', right_on='finess', how='inner' )

In [140]:
df= df.rename(columns={'cat_A' : 'Q1_A', 'cat_B' : 'Q1_B', 'cat_C' : 'Q1_C', 'cat_D' : 'Q1_D'})
df=df.drop(['ind','finess'], axis=1)

In [141]:
df

,Numero_finess,Points_ISA,stjr,dep,Departement,COMINSEE,Capital_Lit_MCO,Capital_Bloc,Capital_imagerie,Capital_Examens_biologiques,...,Dep Moyenne Salaires Horaires,Dep Moyenne Salaires Cadre Horaires,Dep Moyenne Salaires Prof Intermédiaire Horaires,Dep Moyenne Salaires Employé Horaires,Moyenne Revenus Fiscaux Départementaux,Bloc_politique,Q1_A,Q1_B,Q1_C,Q1_D
0,060000528,356892.030000,3,06,Alpes-Maritimes,06088,128.0,8.0,9.0,909.0,...,12.579245,22.518868,12.864151,8.835849,12288.0,Centre droit à droite,70.000000,30.000000,NaN,NaN
1,060780947,174552.540000,3,06,Alpes-Maritimes,06088,105.0,5.0,4.0,0.0,...,12.579245,22.518868,12.864151,8.835849,12288.0,Centre droit à droite,14.545455,38.181818,32.727273,14.545455
2,060791811,146432.633333,3,06,Alpes-Maritimes,06088,91.0,2.0,2.0,0.0,...,12.579245,22.518868,12.864151,8.835849,12288.0,Centre droit à droite,14.545455,38.181818,32.727273,14.545455
3,060780442,22842.380000,2,06,Alpes-Maritimes,06088,30.0,6.0,1.0,0.0,...,12.579245,22.518868,12.864151,8.835849,12288.0,Centre droit à droite,18.478261,41.304348,30.434783,9.782609
4,060780715,342051.171667,2,06,Alpes-Maritimes,06088,256.0,26.0,10.0,0.0,...,12.579245,22.518868,12.864151,8.835849,12288.0,Centre droit à droite,18.478261,41.304348,30.434783,9.782609
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1142,120780085,45665.700000,1,12,Aveyron,12089,34.0,5.0,3.0,235068.0,...,10.115789,20.252632,11.963158,8.478947,9936.0,Extrême gauche à gauche,2.898551,14.975845,48.309179,33.816425
1143,480780097,240863.268333,1,48,Lozère,48095,67.0,4.5,6.0,423436.0,...,10.075000,20.475000,11.775000,8.600000,8555.0,Centre gauche à centre droit,2.898551,14.975845,48.309179,33.816425
1144,480780162,5699.150000,1,48,Lozère,48080,23.0,0.0,1.0,0.0,...,10.075000,20.475000,11.775000,8.600000,8555.0,Centre gauche à centre droit,2.898551,14.975845,48.309179,33.816425
1145,480780121,2396.450000,1,48,Lozère,48140,22.0,0.0,0.0,0.0,...,10.075000,20.475000,11.775000,8.600000,8555.0,Centre droit à droite,2.898551,14.975845,48.309179,33.816425


In [142]:
# On fait la meme chose pour Q8 : 
Qualite_Q8 = Qualite[Qualite['ind']=='Q8']

In [143]:
Qualite_Q8.drop_duplicates(subset ='finess', keep = 'first', inplace=True)

<ipython-input-143-459cf48fd0cf>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Qualite_Q8.drop_duplicates(subset ='finess', keep = 'first', inplace=True)


In [144]:
df8 =pd.merge (Var_CTRL,Qualite_Q8, left_on='Numero_finess', right_on='finess', how='inner' )

In [145]:
df8= df8.rename(columns={'cat_A' : 'Q8_A', 'cat_B' : 'Q8_B', 'cat_C' : 'Q8_C', 'cat_D' : 'Q8_D'})
df8=df8.drop(['ind','finess'], axis=1)

In [146]:
df8

,Numero_finess,Points_ISA,stjr,dep,Departement,COMINSEE,Capital_Lit_MCO,Capital_Bloc,Capital_imagerie,Capital_Examens_biologiques,...,Dep Moyenne Salaires Horaires,Dep Moyenne Salaires Cadre Horaires,Dep Moyenne Salaires Prof Intermédiaire Horaires,Dep Moyenne Salaires Employé Horaires,Moyenne Revenus Fiscaux Départementaux,Bloc_politique,Q8_A,Q8_B,Q8_C,Q8_D
0,060000528,356892.030000,3,06,Alpes-Maritimes,06088,128.0,8.0,9.0,909.0,...,12.579245,22.518868,12.864151,8.835849,12288.0,Centre droit à droite,33.333333,28.571429,38.095238,NaN
1,060780947,174552.540000,3,06,Alpes-Maritimes,06088,105.0,5.0,4.0,0.0,...,12.579245,22.518868,12.864151,8.835849,12288.0,Centre droit à droite,19.852941,21.323529,58.823529,NaN
2,060791811,146432.633333,3,06,Alpes-Maritimes,06088,91.0,2.0,2.0,0.0,...,12.579245,22.518868,12.864151,8.835849,12288.0,Centre droit à droite,19.852941,21.323529,58.823529,NaN
3,060780442,22842.380000,2,06,Alpes-Maritimes,06088,30.0,6.0,1.0,0.0,...,12.579245,22.518868,12.864151,8.835849,12288.0,Centre droit à droite,42.766296,19.236884,37.996820,NaN
4,060780715,342051.171667,2,06,Alpes-Maritimes,06088,256.0,26.0,10.0,0.0,...,12.579245,22.518868,12.864151,8.835849,12288.0,Centre droit à droite,42.766296,19.236884,37.996820,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1142,120780085,45665.700000,1,12,Aveyron,12089,34.0,5.0,3.0,235068.0,...,10.115789,20.252632,11.963158,8.478947,9936.0,Extrême gauche à gauche,6.018519,10.339506,83.641975,NaN
1143,480780097,240863.268333,1,48,Lozère,48095,67.0,4.5,6.0,423436.0,...,10.075000,20.475000,11.775000,8.600000,8555.0,Centre gauche à centre droit,6.018519,10.339506,83.641975,NaN
1144,480780162,5699.150000,1,48,Lozère,48080,23.0,0.0,1.0,0.0,...,10.075000,20.475000,11.775000,8.600000,8555.0,Centre gauche à centre droit,6.018519,10.339506,83.641975,NaN
1145,480780121,2396.450000,1,48,Lozère,48140,22.0,0.0,0.0,0.0,...,10.075000,20.475000,11.775000,8.600000,8555.0,Centre droit à droite,6.018519,10.339506,83.641975,NaN


In [147]:
df2 = pd.merge(df,df8)

In [148]:
df2

,Numero_finess,Points_ISA,stjr,dep,Departement,COMINSEE,Capital_Lit_MCO,Capital_Bloc,Capital_imagerie,Capital_Examens_biologiques,...,Moyenne Revenus Fiscaux Départementaux,Bloc_politique,Q1_A,Q1_B,Q1_C,Q1_D,Q8_A,Q8_B,Q8_C,Q8_D
0,060000528,356892.030000,3,06,Alpes-Maritimes,06088,128.0,8.0,9.0,909.0,...,12288.0,Centre droit à droite,70.000000,30.000000,NaN,NaN,33.333333,28.571429,38.095238,NaN
1,060780947,174552.540000,3,06,Alpes-Maritimes,06088,105.0,5.0,4.0,0.0,...,12288.0,Centre droit à droite,14.545455,38.181818,32.727273,14.545455,19.852941,21.323529,58.823529,NaN
2,060791811,146432.633333,3,06,Alpes-Maritimes,06088,91.0,2.0,2.0,0.0,...,12288.0,Centre droit à droite,14.545455,38.181818,32.727273,14.545455,19.852941,21.323529,58.823529,NaN
3,060780442,22842.380000,2,06,Alpes-Maritimes,06088,30.0,6.0,1.0,0.0,...,12288.0,Centre droit à droite,18.478261,41.304348,30.434783,9.782609,42.766296,19.236884,37.996820,NaN
4,060780715,342051.171667,2,06,Alpes-Maritimes,06088,256.0,26.0,10.0,0.0,...,12288.0,Centre droit à droite,18.478261,41.304348,30.434783,9.782609,42.766296,19.236884,37.996820,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1142,120780085,45665.700000,1,12,Aveyron,12089,34.0,5.0,3.0,235068.0,...,9936.0,Extrême gauche à gauche,2.898551,14.975845,48.309179,33.816425,6.018519,10.339506,83.641975,NaN
1143,480780097,240863.268333,1,48,Lozère,48095,67.0,4.5,6.0,423436.0,...,8555.0,Centre gauche à centre droit,2.898551,14.975845,48.309179,33.816425,6.018519,10.339506,83.641975,NaN
1144,480780162,5699.150000,1,48,Lozère,48080,23.0,0.0,1.0,0.0,...,8555.0,Centre gauche à centre droit,2.898551,14.975845,48.309179,33.816425,6.018519,10.339506,83.641975,NaN
1145,480780121,2396.450000,1,48,Lozère,48140,22.0,0.0,0.0,0.0,...,8555.0,Centre droit à droite,2.898551,14.975845,48.309179,33.816425,6.018519,10.339506,83.641975,NaN


In [149]:
df2.columns

Index(['Numero_finess', 'Points_ISA', 'stjr', 'dep', 'Departement', 'COMINSEE',
       'Capital_Lit_MCO', 'Capital_Bloc', 'Capital_imagerie',
       'Capital_Examens_biologiques', 'Capital_telemed', 'SEJHC_MED',
       'SEJHC_CHI', 'SEJHC_OBS', 'SEJHC_MCO', 'Taux_chomage_par_dep',
       'Population', 'Dep Moyenne Salaires Horaires',
       'Dep Moyenne Salaires Cadre Horaires',
       'Dep Moyenne Salaires Prof Intermédiaire Horaires',
       'Dep Moyenne Salaires Employé Horaires',
       'Moyenne Revenus Fiscaux Départementaux', 'Bloc_politique', 'Q1_A',
       'Q1_B', 'Q1_C', 'Q1_D', 'Q8_A', 'Q8_B', 'Q8_C', 'Q8_D'],
      dtype='object')

In [150]:
liste1 = df2.Numero_finess.unique()
liste2 = Var_CTRL.Numero_finess.unique() 
res = [value for value in liste2 if value not in liste1]
res

['130045263',
 '310021571',
 '540000056',
 '690041132',
 '690041124',
 '750810798',
 '070007927',
 '230780074',
 '330060658',
 '560029068',
 '560002974',
 '570000141',
 '640019220',
 '640780862',
 '670020098']

In [151]:
df_res = Var_CTRL[Var_CTRL.Numero_finess.isin(res)]
df_res[[ 'Q1_A', 'Q1_B', 'Q1_C', 'Q1_D', 'Q8_A', 'Q8_B', 'Q8_C','Q8_D']]=np.nan

C:\Users\louis\anaconda3\lib\site-packages\pandas\core\indexing.py:659: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[k] = np.nan
<ipython-input-151-c0d3f17da80e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_res[[ 'Q1_A', 'Q1_B', 'Q1_C', 'Q1_D', 'Q8_A', 'Q8_B', 'Q8_C','Q8_D']]=np.nan
C:\Users\louis\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = valu

In [152]:
Var_CTRL= pd.concat([df2, df_res])

In [153]:
Var_CTRL

,Numero_finess,Points_ISA,stjr,dep,Departement,COMINSEE,Capital_Lit_MCO,Capital_Bloc,Capital_imagerie,Capital_Examens_biologiques,...,Moyenne Revenus Fiscaux Départementaux,Bloc_politique,Q1_A,Q1_B,Q1_C,Q1_D,Q8_A,Q8_B,Q8_C,Q8_D
0,060000528,356892.030000,3,06,Alpes-Maritimes,06088,128.0,8.0,9.0,909.0,...,12288.0,Centre droit à droite,70.000000,30.000000,NaN,NaN,33.333333,28.571429,38.095238,NaN
1,060780947,174552.540000,3,06,Alpes-Maritimes,06088,105.0,5.0,4.0,0.0,...,12288.0,Centre droit à droite,14.545455,38.181818,32.727273,14.545455,19.852941,21.323529,58.823529,NaN
2,060791811,146432.633333,3,06,Alpes-Maritimes,06088,91.0,2.0,2.0,0.0,...,12288.0,Centre droit à droite,14.545455,38.181818,32.727273,14.545455,19.852941,21.323529,58.823529,NaN
3,060780442,22842.380000,2,06,Alpes-Maritimes,06088,30.0,6.0,1.0,0.0,...,12288.0,Centre droit à droite,18.478261,41.304348,30.434783,9.782609,42.766296,19.236884,37.996820,NaN
4,060780715,342051.171667,2,06,Alpes-Maritimes,06088,256.0,26.0,10.0,0.0,...,12288.0,Centre droit à droite,18.478261,41.304348,30.434783,9.782609,42.766296,19.236884,37.996820,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
439,560002974,5098.133333,3,56,Morbihan,56162,10.0,0.0,0.0,0.0,...,12381.0,Centre gauche à centre droit,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
468,570000141,7607.485000,1,57,Moselle,57631,24.0,0.0,0.0,0.0,...,10926.0,Droite à extrême droite,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
635,640019220,75597.775000,2,64,Pyrénées-Atlantiques,64445,137.0,19.0,0.0,0.0,...,10638.0,Centre gauche à centre droit,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
638,640780862,413.120000,1,64,Pyrénées-Atlantiques,64445,0.0,0.0,0.0,0.0,...,10638.0,Centre gauche à centre droit,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
